In [ ]:
import pandas as pd
import numpy as np
import datetime
from pyexcelerate import Workbook
import warnings
from decimal import Decimal

import calendar
import math
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 


In [ ]:
def save_backup(path_backup,arrs):
    try:
        backup = pd.HDFStore(path_backup)
        for arr in arrs:
            backup[str(arr.name)] = arr
        backup.close()   
    except:
        backup.close()

# типизируем cтолбцы таблица,
# type_ к какому типу приводим, 
# mask по каком услову в столбце типизируем
def retype_multiindex(df , type_, mask, levelForFindMask):
    for col in df.columns:
        try:
            if col[levelForFindMask].find(mask)!=-1:
                df.loc[:,col] = df.loc[:,col].fillna(0).astype('int').astype(type_)
        except AttributeError:
            pass
        except ValueError:
            pass
    return df

def retype_index(df, type_ , mask):
    for col in df.columns:
        try:
            if col.find(mask)!=-1:
                df.loc[:,col] = df.loc[:,col].fillna(0).astype(type_)
        except AttributeError:
            pass
        except ValueError:
            pass
    return df
    

# Упаковка справочника для insert vpr
def rev_to_dict(ind,find):
#     ind = ind.astype('str')
    df = pd.DataFrame(index=ind,data=find.values,columns=[find.name])
    df = df[(df.index!=0)|(pd.isnull(df.index)==True)]
    df = df.groupby(df.index).first()
    dictt =  df.to_dict('index')
    return dictt  


# Аналог впр
def insert_vpr(x,dictt,name):
    try:
        return dictt[x][name]
    except KeyError:
        return 0
    
# Собириает текущие данные
def get_current_data(data, global_index, current_month):
    index = [x for x in data.columns if x in global_index]
    index.extend([current_month])
    return data[index]


def save_pyex(path,df):
    wb = Workbook()
    wb.new_sheet('sheet1',data =  [df.columns.tolist(),]+df.values.tolist())
    wb.save(path) 


    
# def dict_keys(mapping):
#     cons = pd.DataFrame()
#     for ind, col in enumerate(mapping.columns):
#         try:
#             frame_ = mapping.loc[:, (col[0],['Артикул','Часть'])]
#             frame_.columns = ['Артикул','Часть']
#             cons = cons.append(frame_)
#         except KeyError:
#             pass
#         except ValueError:
#             pass
        
#     return cons.drop_duplicates()


def dict_keys(col_1_val,mapping):
    cons = pd.DataFrame()
    try:
        frame_ = mapping.loc[:, (col_1_val,['Артикул','Часть'])]
        frame_.columns = ['Артикул','Часть']
        cons = cons.append(frame_)
    except KeyError:
        pass
    except ValueError:
        pass
        
    return cons.drop_duplicates()


def dict_keys_free(col_1_val,mapping,arr_cols):
    cons = pd.DataFrame()
    try:
        frame_ = mapping.loc[:, (col_1_val,arr_cols)]
        frame_.columns = arr_cols
        cons = cons.append(frame_)
    except KeyError:
        pass
    except ValueError:
        pass
        
    return cons.drop_duplicates()

# Установить тип продукции для 3 передела
def get_type_product(data):
    colName = 'Тип продукции'
    data[colName] = ''
    for art in data['Артикул'].unique():
        min_index = min(list(data[data['Артикул']==art].index)  ) 
        max_index  = max(list(data[data['Артикул']==art].index)  ) 
#         data.loc[min_index,colName] = 'готовая продукция'
        data.loc[max_index,colName] = 'основная продукция'
    return data


# ['Затраты на Индив. пакет, признак', \
# 'Затраты на Подложку, признак', \
# 'Затраты на Групп. Пакет, признак'],
# current_month,
# 'Норма упаковки, кг/чел*час',
# NZ_UKPF,
# ['индив. пакет', 'подложка', 'групповой пакет'],
# 'Участок фасовки и упаковки '

# Установка новых % в накладные расходы по упаковке
def input_proc_packaging(df,arr_priznak, date, filter_, df2,  arr_priznak1 ,filter_2):
    cols1 = ('downstream', 'Упак', arr_priznak1[0])
    cols2 = ('downstream', 'Упак', arr_priznak1[1])
    cols3 = ('downstream', 'Упак', arr_priznak1[2])
    cols_all = ('downstream', 'Упак', 'всего')
    
    all_cost = df2.loc[df2['Месяц']['Месяц']['Месяц']==date, (cols_all)].sum()
    
    d1 = df[df[arr_priznak[0]] == "да"][date].sum() / \
    df2[df2['Наименование']['Наименование']['Наименование']==filter_].loc[:,cols1].sum()
    
    d2 = df[df[arr_priznak[1]] == "да"][date].sum() / \
    df2[df2['Наименование']['Наименование']['Наименование']==filter_].loc[:,cols2].sum()
    
    d3 = df[df[arr_priznak[2]] == "да"][date].sum() / \
    df2[df2['Наименование']['Наименование']['Наименование']==filter_].loc[:,cols3].sum()
    
    df2.loc[df2['Наименование']['Наименование']['Наименование']==filter_2,(cols1)] = d1/(d1+d2+d3)
    df2.loc[df2['Наименование']['Наименование']['Наименование']==filter_2,(cols2)] = d2/(d1+d2+d3)
    df2.loc[df2['Наименование']['Наименование']['Наименование']==filter_2,(cols3)] = d3/(d1+d2+d3)
    
    #     Подставляем значения всего умноженные на эти проценты
    
    df2.loc[df2['Месяц']['Месяц']['Месяц']==date,(cols1)] = d1/(d1+d2+d3) * all_cost
    df2.loc[df2['Месяц']['Месяц']['Месяц']==date,(cols2)] = d2/(d1+d2+d3) * all_cost
    df2.loc[df2['Месяц']['Месяц']['Месяц']==date,(cols3)] = d3/(d1+d2+d3) * all_cost
    


    return df2

def save_iter_month_xlsx(directory_out,df,months,name):
    try:
        for date in iter_months:
            df[date].to_excel(directory_out+str(date.month)+name)
    except KeyError:
        pass
    
# Функция Реструктурирует таблицу с мультиколонками  для того чтобы pandas воспринимал индекс   
def restruct_multitindex(df):
    df.rename(columns=dict((x[1],'') for x in df.columns if x[1].find('Unnamed')!=-1) ,inplace=True)
    df = df.dropna(axis='index', how='any', subset=[('Артикул','','')])
    df  = retype_multiindex(df, 'int', 'Артикул', 0)
    df = df.set_index('Артикул')
    return df


# Функции для django
def replace_to_correct_datetime(df):
    l = list(df.columns)
    
    for ind, col in enumerate(df.columns):
        try:
            l[ind] = datetime.datetime.fromtimestamp((int(l[ind]) - 25569) * 86400)
        except (TypeError,ValueError):
            pass
    
    df.columns = l
    
    return df


def replace_to_correct_datetime1(df):
    l = list(df.columns)
    
    for ind, col in enumerate(df.columns):
        l[ind] = list(l[ind])
        for ind1, el in enumerate(l[ind]):
            try:
                l[ind][ind1] = datetime.datetime.fromtimestamp((int(l[ind][ind1]) - 25569) * 86400)
            except (TypeError,ValueError):
                pass
            
        if ind1 == len(l[ind])-1:
            l[ind] = tuple(l[ind])
            
    
    df.columns = pd.MultiIndex.from_tuples(l)
            
    return df


In [ ]:
directory = 'C:\\Users\\Кедрун Никита\\PycharmProjects\\akz_\\input\\'
filename = 'УКПФ.xlsx'
mapping = 'Mapping.xlsx'
coef_cenn =               'Коэффициент ценности.xlsx'
ost_nach_g =             'Остатки на начало года УКПФ.xlsx'

In [ ]:
directory = 'C:\\Users\\Кедрун Никита\\PycharmProjects\\akz_\\input\\'
filename = 'МПФ.xlsx'
mapping = 'Mapping.xlsx'
coef_cenn =               'Коэффициент ценности.xlsx'
ost_nach_g =             'Остатки на начало года МПФ.xlsx'

In [ ]:
global_index = ['Артикул', 'Продукция', 'Номенклатура', 'Канал', 'Тип']
cons_pr = {}
sale_finished_products_UKPF =  pd.read_excel(directory+filename,sheet_name='Продажи ГП')
balance_depot_stock_UKPF =  pd.read_excel(directory+filename,sheet_name='Остатки')
adm_UKPF =  pd.read_excel(directory+filename,sheet_name='Адм')
PP_UKPF =  pd.read_excel(directory+filename,sheet_name='РР')
per_3_UKPF =  pd.read_excel(directory+filename,sheet_name='3 передел')
per_2_UKPF_meat =  pd.read_excel(directory+filename,sheet_name='2 передел мясо')
per_2_UKPF_product =  pd.read_excel(directory+filename,sheet_name='2 передел гп')
per_2_UKPF_losses =  pd.read_excel(directory+filename,sheet_name='2 передел потери')
per_1_UKPF =  pd.read_excel(directory+filename,sheet_name='1 передел')
NZ_UKPF =  pd.read_excel(directory+filename,sheet_name='Накладные затраты',header=[0,1,2])
UIS_UKPF =  pd.read_excel(directory+filename,sheet_name='УиС')
DATA_upload =  pd.read_excel(directory+filename,sheet_name='Общие данные по выходу')
Production_output =  pd.read_excel(directory+filename,sheet_name='Выпуск ГП')
mapping =  pd.read_excel(directory+mapping,sheet_name='Mapping',header=[0,1] )
koef_ef =  pd.read_excel(directory+coef_cenn,sheet_name='Лист1')
ost_start_new_form =  pd.read_excel(directory+ost_nach_g,sheet_name='Sheet1',index_col=0)
mmo =  pd.read_excel(directory+filename,sheet_name='ММО',header=[0,1] )



In [ ]:
ar = [global_index,cons_pr,sale_finished_products_UKPF,balance_depot_stock_UKPF,adm_UKPF,PP_UKPF,per_3_UKPF,per_2_UKPF_meat, per_2_UKPF_product,per_2_UKPF_losses,per_1_UKPF,NZ_UKPF,UIS_UKPF,DATA_upload,Production_output,mapping,koef_ef,ost_start_new_form,mmo]

In [ ]:
def get_ssmp_ukpf(directory,filename,mapping,coef_cenn,ost_nach_g,mon,global_index):
# def get_ssmp_ukpf(ar,mon):
#     global_index = ar[0]
#     cons_pr = ar[1]
#     sale_finished_products_UKPF =  ar[2]
#     balance_depot_stock_UKPF =  ar[3]
#     adm_UKPF =  ar[4]
#     PP_UKPF =  ar[5]
#     per_3_UKPF =  ar[6]
#     per_2_UKPF_meat =  ar[7]
#     per_2_UKPF_product =  ar[8]
#     per_2_UKPF_losses =  ar[9]
#     per_1_UKPF =  ar[10]
#     NZ_UKPF =  ar[11]
#     UIS_UKPF =  ar[12]
#     DATA_upload =  ar[13]
#     Production_output =  ar[14]
    
#     mapping =  ar[15]
#     koef_ef =  ar[16]
#     ost_start_new_form =  ar[17]
#     mmo = ar[18]

#    Структуры для хранения данных в выходных объектах

    cons_pr = {}
    per_1 = {}
    per_2 = {}
    sale_finished_products_UKPF =  pd.read_excel(directory+filename,sheet_name='Продажи ГП')
    balance_depot_stock_UKPF =  pd.read_excel(directory+filename,sheet_name='Остатки')
    adm_UKPF =  pd.read_excel(directory+filename,sheet_name='Адм')
    PP_UKPF =  pd.read_excel(directory+filename,sheet_name='РР')
    per_3_UKPF =  pd.read_excel(directory+filename,sheet_name='3 передел')
    per_2_UKPF_meat =  pd.read_excel(directory+filename,sheet_name='2 передел мясо')
    per_2_UKPF_product =  pd.read_excel(directory+filename,sheet_name='2 передел гп')
    per_2_UKPF_losses =  pd.read_excel(directory+filename,sheet_name='2 передел потери')
    per_1_UKPF =  pd.read_excel(directory+filename,sheet_name='1 передел')
    NZ_UKPF =  pd.read_excel(directory+filename,sheet_name='Накладные затраты',header=[0,1,2])
    UIS_UKPF =  pd.read_excel(directory+filename,sheet_name='УиС')
    DATA_upload =  pd.read_excel(directory+filename,sheet_name='Общие данные по выходу')
    Production_output =  pd.read_excel(directory+filename,sheet_name='Выпуск ГП')
    mapping =  pd.read_excel(directory+mapping,sheet_name='Mapping',header=[0,1] )
    koef_ef =  pd.read_excel(directory+coef_cenn,sheet_name='Лист1')
    ost_start_new_form =  pd.read_excel(directory+ost_nach_g,sheet_name='Sheet1',index_col=0)
    mmo =  pd.read_excel(directory+filename,sheet_name='ММО',header=[0,1] )
    
    
    
    
        # Блок типизации данных
    dates = [x for x in PP_UKPF.columns if isinstance( x, datetime.datetime)]  

    # Изменил тип артикула
    mapping  = retype_multiindex(mapping, 'int', 'Артикул', 1)


    # Изменил тип артикула

    per_1_UKPF  = retype_index(per_1_UKPF, 'int', 'Артикул')
    per_2_UKPF_meat  = retype_index(per_2_UKPF_meat, 'int', 'Артикул')
    per_2_UKPF_product  = retype_index(per_2_UKPF_product, 'int', 'Артикул')
    per_2_UKPF_losses  = retype_index(per_2_UKPF_losses, 'int', 'Артикул')
    per_3_UKPF  = retype_index(per_3_UKPF, 'int', 'Артикул')
    Production_output  = retype_index(Production_output, 'int', 'Артикул')
    UIS_UKPF  = retype_index(UIS_UKPF, 'int', 'Артикул')
    sale_finished_products_UKPF  = retype_index(sale_finished_products_UKPF, 'int', 'Артикул')
    balance_depot_stock_UKPF  = retype_index(balance_depot_stock_UKPF, 'int', 'Артикул')
    mmo = restruct_multitindex(mmo)
    
    iter_months = [datetime.datetime(year_report,x,calendar.monthrange(year_report,x)[1],0,0) for x in range(1,13)]
    
    for ind___, current_month in enumerate(iter_months):
        
#         нужна для формирования таблицы с остатками
        if ind___>0:
            prev_month = iter_months[ind___-1]
        
        print(current_month)
        # получил текущие даныные по current_month
        curr_per_1_UKPF = get_current_data(per_1_UKPF, global_index, current_month)
        curr_per_2_UKPF_meat = get_current_data(per_2_UKPF_meat, global_index, current_month)
        curr_per_2_UKPF_product = get_current_data(per_2_UKPF_product, global_index, current_month)
        curr_per_2_UKPF_losses = get_current_data(per_2_UKPF_losses, global_index, current_month)
        curr_per_3_UKPF = get_current_data(per_3_UKPF.set_index(['Артикул','Тип продукции']), global_index, current_month).reset_index()
        curr_Production_output_UKPF = get_current_data(Production_output, global_index, current_month)
        curr_UIS_UKPF = get_current_data(UIS_UKPF, global_index, current_month)
        curr_sale_finished_products_UKPF = get_current_data(sale_finished_products_UKPF, global_index, current_month)
        curr_balance_depot_stock_UKPF = get_current_data(balance_depot_stock_UKPF, global_index, current_month)
        curr_mmo = get_current_data(mmo, global_index, current_month).reset_index()

        # отсекли только то что в соответствующих переделах из справочников mapping
        
        
        # Блок получения данных по mapping (Отсекаем только то что есть в mapping)
        
        key = filename.split('.')[0]
        dict_mapping = {'УКПФ':['Мэппинг УКПФ 1 передел','Мэппинг УКПФ 2 передел','Мэппинг УКПФ Затраты'],
        'МПФ':['Мэппинг МПФ 1 передел','Мэппинг МПФ 2 передел','Мэппинг МПФ Затраты']}
        
        
        cons_1_per = curr_per_1_UKPF[curr_per_1_UKPF['Артикул'].isin(mapping[mapping[dict_mapping[key][0]]['Артикул']!=0][dict_mapping[key][0]]['Артикул'])].groupby(['Артикул','Продукция']).sum().reset_index()
#         cons_2_per_meat = curr_per_2_UKPF_meat[curr_per_2_UKPF_meat['Артикул'].isin(mapping[mapping[dict_mapping[key][1]]['Артикул']!=0][dict_mapping[key][1]]['Артикул'])].groupby(['Артикул','Продукция']).sum().reset_index()
        cons_2_per_meat = curr_per_2_UKPF_meat.copy()
        cons_2_per_product = curr_per_2_UKPF_product[curr_per_2_UKPF_product['Артикул'].isin(mapping[mapping[dict_mapping[key][1]]['Артикул']!=0][dict_mapping[key][1]]['Артикул'])].groupby(['Артикул','Продукция']).sum().reset_index()
        cons_2_per_losses = curr_per_2_UKPF_losses[~curr_per_2_UKPF_losses['Артикул'].apply(lambda x: isinstance(x, str))]
        
        
        
        


        dict_chast_1 = rev_to_dict(dict_keys(dict_mapping[key][0],mapping)['Артикул'],dict_keys(dict_mapping[key][0],mapping)['Часть'])
        # Присваем части
        cons_1_per['часть'] = cons_1_per.apply(lambda x: insert_vpr(x['Артикул'],dict_chast_1,'Часть') ,axis = 1).astype('str')
        
        
        # 2 передел для анализа

        dict_chast_2 = rev_to_dict(dict_keys(dict_mapping[key][1],mapping)['Артикул'],dict_keys(dict_mapping[key][1],mapping)['Часть'])

        curr_per_2_UKPF_meat['часть'] = curr_per_2_UKPF_meat.apply(lambda x: insert_vpr(x['Артикул'],dict_chast_2,'Часть') ,axis = 1).astype('str')
        cons_2_per_product['часть'] = cons_2_per_product.apply(lambda x: insert_vpr(x['Артикул'],dict_chast_2,'Часть') ,axis = 1).astype('str')
        cons_2_per_losses['часть'] = cons_2_per_losses.apply(lambda x: insert_vpr(x['Артикул'],dict_chast_2,'Часть') ,axis = 1).astype('str')
        curr_per_3_UKPF['часть'] = curr_per_3_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_chast_2,'Часть') ,axis = 1).astype('str')
        # 3 передел для анализа
#         curr_per_3_UKPF = get_type_product(curr_per_3_UKPF)

        
        dict_chast_3 = rev_to_dict(dict_keys(dict_mapping[key][2],mapping)['Артикул'],dict_keys(dict_mapping[key][2],mapping)['Часть'])
        # Выпуск ГП для анализа
        curr_Production_output_UKPF['часть'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_chast_3,'Часть') ,axis = 1).astype('str')
        
        
        
        
        # группировка по частям
        # Первый передел по частям УКПФ
        part_1_cons_1_per = cons_1_per[['часть',current_month]].groupby('часть').sum().reset_index()
        
       
        
        # объем живка
        V_zhivka = DATA_upload[DATA_upload['Наименование']=='Объем живка, кг'].loc[:,current_month].values[0] 
        #объем первого передела
        V_1_peredel = cons_1_per[current_month].sum() 
        # С/с 1 передела
        SS_1_zperedel_without_amort = DATA_upload[DATA_upload['Наименование']=='Сырье (живая птица) без амортизации'].loc[:,current_month].values[0]/1000 
        # Средняя с/с 1 передела, без амортизации
        Mean_SS_1_zperedel_without_amort  = SS_1_zperedel_without_amort/V_1_peredel * 1000
        
        
        part_1_cons_1_per['%%от тушки'] = part_1_cons_1_per.apply(lambda x: x[current_month]/V_zhivka,axis = 1)


        dict_koef_ef = rev_to_dict(koef_ef[koef_ef['Признак']=='убой']['Продукция'],koef_ef[koef_ef['Признак']=='убой']['Коэффициент'])
        part_1_cons_1_per['Доля стоимости 1 передела'] = part_1_cons_1_per.apply(lambda x: insert_vpr(x['часть'],dict_koef_ef,'Коэффициент') ,axis = 1).astype('str')
        
        
        
        part_1_cons_1_per['усл.ед.'] = part_1_cons_1_per['Доля стоимости 1 передела'].astype('float64') * part_1_cons_1_per['%%от тушки']
        
        part_1_cons_1_per['Распределение с/с 1 передела'] = part_1_cons_1_per.apply(lambda x: SS_1_zperedel_without_amort * x['усл.ед.'] / part_1_cons_1_per['усл.ед.'].sum() ,axis = 1)


        part_1_cons_1_per['С/с 1 передела'] = part_1_cons_1_per.apply(lambda x: 0 if  x[current_month] ==0 else  x['Распределение с/с 1 передела'] / x[current_month] * 1000 ,axis = 1)

        
        # Конец первого передела
        

        
        # Начало второго передела
#         part_1_cons_2_per_meat = cons_2_per_meat[['часть',current_month]].groupby('часть').sum().reset_index()

        part_1_cons_2_per_product = cons_2_per_product[['часть',current_month]].groupby('часть').sum().reset_index()

        #Объем на разделку без потерь
        V_raz_without_losses = cons_2_per_meat[current_month].sum() 


        # С/с тушки, без амортизации

        SS_tushki_without_amort = part_1_cons_1_per[part_1_cons_1_per['часть']=='Тушка']['С/с 1 передела'].values[0]

        # С/с 2 передела, без амортизации
        Mean_SS_1_zperedel_without_amort = V_raz_without_losses * SS_tushki_without_amort / 1000

        part_1_cons_2_per_product['%%от тушки'] = part_1_cons_2_per_product.apply(lambda x: x[current_month]/V_raz_without_losses,axis = 1)
        
        


        dict_koef_ef = rev_to_dict(koef_ef[koef_ef['Признак']=='обвалка']['Продукция'],koef_ef[koef_ef['Признак']=='обвалка']['Коэффициент'])
        part_1_cons_2_per_product['Доля стоимости 2 передела'] = part_1_cons_2_per_product.apply(lambda x: insert_vpr(x['часть'],dict_koef_ef,'Коэффициент') ,axis = 1)


        part_1_cons_2_per_product['усл.ед.'] = part_1_cons_2_per_product['Доля стоимости 2 передела'].astype('float64') * part_1_cons_2_per_product['%%от тушки']

        part_1_cons_2_per_product['Распределение с/с 2 передела'] = part_1_cons_2_per_product.apply(lambda x: Mean_SS_1_zperedel_without_amort * x['усл.ед.'] / part_1_cons_2_per_product['усл.ед.'].sum() ,axis = 1)

        part_1_cons_2_per_product['С/с 2 передела'] = part_1_cons_2_per_product.apply(lambda x: 0 if  x[current_month] ==0 else  x['Распределение с/с 2 передела'] / x[current_month] * 1000 ,axis = 1)
        
#         if ind___+1==1:
#             return curr_Production_output_UKPF
#             return part_1_cons_1_per,part_1_cons_2_per_product
        
        # логика по чахохбили df2 = данные со второго передела df3 данные с 3 передела
        
        def get_chahoh(df_2,df_3):
            if key =='УКПФ':
                return ( df_2[df_2['часть']=='грудка']['С/с 2 передела'].sum()*df_3[df_3['Продукция']=='Грудка ЦБ'][current_month].sum()+ \
                         df_2[df_2['часть']=='окорочок']['С/с 2 передела'].sum()*df_3[df_3['Продукция']=='Окорочок ЦБ'][current_month].sum() ) / \
                        ( df_3[df_3['Продукция']=='Грудка ЦБ'][current_month].sum() + df_3[df_3['Продукция']=='Окорочок ЦБ'][current_month].sum())
            elif key=='МПФ':
                
                return  ( df_2[df_2['часть']=='грудка']['С/с 2 передела'].sum()*df_3[df_3['Продукция']=='Грудка ЦБ в групповой упаковке в оборотной таре (охл.)'][current_month].sum()+ \
                         df_2[df_2['часть']=='окорочок']['С/с 2 передела'].sum()*df_3[df_3['Продукция']=='Окорочок ЦБ в групповой упаковке в оборотной таре (охл.)'][current_month].sum() ) / \
                        ( df_3[df_3['Продукция']=='Окорочок ЦБ в групповой упаковке в оборотной таре (охл.)'][current_month].sum() + df_3[df_3['Продукция']=='Грудка ЦБ в групповой упаковке в оборотной таре (охл.)'][current_month].sum())
        
        # логика по ммо df1 это текущие данные ммо(исходник), df2 это рассчитанные данные по второму переделу
#         if ind___+1==1:
#             return get_chahoh(part_1_cons_2_per_product,curr_per_3_UKPF)
        df_for_mo_mapping = pd.concat([curr_per_1_UKPF,
                           curr_per_2_UKPF_meat,
                           curr_per_2_UKPF_product,
                           curr_per_2_UKPF_losses,
                           curr_per_3_UKPF])[['Артикул',current_month]]
        
        
        
        
        def get_mmo(df,df_for_mo_mapping):
            df = df.set_index('Артикул')[current_month].reset_index()
            df['часть'] = df.apply(lambda x: insert_vpr(x['Артикул'],dict_chast_2,'Часть') ,axis = 1).astype('str')
            
            
            df['данные по переделам'] = df.apply(lambda x: df_for_mo_mapping[df_for_mo_mapping['Артикул'] == x['Артикул']][current_month].sum() ,axis = 1)
            
            vals_for_mmo = part_1_cons_1_per.rename(columns = {'С/с 1 передела':'С/с 2 передела'})[['часть','С/с 2 передела']].\
                                                append(part_1_cons_2_per_product[['часть','С/с 2 передела']])
            
            vals_ = rev_to_dict(vals_for_mmo['часть'],vals_for_mmo['С/с 2 передела'])
            
            
            
            df['стоимость с/с'] = df.apply(lambda x: insert_vpr(x['часть'],vals_,'С/с 2 передела') ,axis = 1)
            
            df['С/с Птицеводства, тыс. тг'] = df['Объем затрат'] * df['стоимость с/с'] / 1000
            
            val_mmo = curr_Production_output_UKPF[curr_Production_output_UKPF['Артикул']==117][current_month].sum()
            
            return df['С/с Птицеводства, тыс. тг'].sum()/val_mmo * 1000
  
        part_1_cons_2_per_product = part_1_cons_2_per_product.append(pd.DataFrame(columns=['часть','С/с 2 передела'],data = {'часть':['Чахохбили'],'С/с 2 передела':[get_chahoh(part_1_cons_2_per_product,curr_per_3_UKPF)] }))
        part_1_cons_2_per_product = part_1_cons_2_per_product.append(pd.DataFrame(columns=['часть','С/с 2 передела'],data = {'часть':['ММО'],'С/с 2 передела':[get_mmo(curr_mmo,df_for_mo_mapping)] }))
        
    #         ///////////////////////////////////////////////////////////////////////
        
#         if ind___+1==1:
#             return part_1_cons_2_per_product
# #             return get_mmo(curr_mmo,df_for_mo_mapping)
        

        

        
        per_1[current_month] = part_1_cons_1_per
        per_2[current_month] = part_1_cons_2_per_product
        
        
        #  -----------------------       
        
        
        # ПГ_CC МП

        def func(x, col, per1, per2_1, per2_2, per2_3, per3):
            filter_ = 'Артикул'
            if per3[(per3[filter_] == x[filter_])&(per3['Тип продукции'] == 'основная продукция')][col].sum() >0:
                return per3[(per3[filter_] == x[filter_])&(per3['Тип продукции'] == 'основная продукция')][col].sum()
            else:
                return x[current_month]
#             sum_ = per1[per1[filter_] == x[filter_]][col].sum() + \
#                 per2_1[per2_1[filter_] == x[filter_]][col].sum() + \
#                 per2_2[per2_2[filter_] == x[filter_]][col].sum()+ \
#                 per2_3[per2_3[filter_] == x[filter_]][col].sum()+ \
#                 per3[(per3[filter_] == x[filter_])&(per3['Тип продукции'] == 'основная продукция')][col].sum()
#             return sum_ if sum_>0 else x[col]
        
        
         
        
        curr_Production_output_UKPF['Объем мяса  (без воды и специй),кг'] = \
        curr_Production_output_UKPF.apply(lambda x:func(x,current_month, cons_1_per, cons_2_per_meat, cons_2_per_product, \
        cons_2_per_losses, curr_per_3_UKPF) ,axis = 1)
        
#         if ind___+1==1:
#             return curr_Production_output_UKPF
        
       
        
        # ------------------------------

        df_sebes_group = part_1_cons_2_per_product[['часть','С/с 2 передела']].rename(columns={'С/с 2 передела': 'Себес'}).\
        append( part_1_cons_1_per[['часть','С/с 1 передела']].rename(columns={'С/с 1 передела': 'Себес'}) ).\
        groupby('часть').sum().reset_index()

        dict_sebes = rev_to_dict(df_sebes_group['часть'],df_sebes_group['Себес'])
        curr_Production_output_UKPF['Мясосырье, Птицевосдтво, тг/кг'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['часть'],dict_sebes,'Себес') ,axis = 1)

        # ------------------------------
        
        
        curr_Production_output_UKPF['C/c мясосырья, Птицеводство тыс. тг'] = curr_Production_output_UKPF.apply(lambda x: x['Объем мяса  (без воды и специй),кг'] * \
                                         x['Мясосырье, Птицевосдтво, тг/кг'] / 1000 ,axis = 1)
        
        
        
        
        # ------------------------------

        curr_Production_output_UKPF['C/c мясосырья, Птицеводство тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['C/c мясосырья, Птицеводство тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)
        
        
        
        # ------------------------------

        def func(x, col, uis, mask):
            filter_ = 'Артикул'
            sum_ = uis[(uis[filter_] == x[filter_]) & (uis['Номенклатура'] == mask)][col].sum()
            return sum_/1000

        curr_Production_output_UKPF['Специи, добавки тыс. тг'] = \
        curr_Production_output_UKPF.apply(lambda x:func(x,current_month, curr_UIS_UKPF ,'Специи, добавки') ,axis = 1)

        # ------------------------------

        curr_Production_output_UKPF['Специи, добавки тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Специи, добавки тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)

        # ------------------------------
        
        
        if key =='УКПФ':
            
            curr_Production_output_UKPF['Упаковочный материал тыс. тг'] = \
            curr_Production_output_UKPF.apply(lambda x:func(x,current_month, curr_UIS_UKPF, 'Упаковочный материал') ,axis = 1)
        
        elif key=='МПФ':
            
            all_zat = curr_UIS_UKPF[curr_UIS_UKPF['Номенклатура']=='Упаковочный материал'][current_month].sum()
            curr_UIS_UKPF['mapping'] = curr_UIS_UKPF.apply(lambda x: "ГП"
                                                           if curr_Production_output_UKPF[curr_Production_output_UKPF['Артикул'] == x['Артикул']][current_month].sum() > 0 
                                                           else "промпереработка"  ,axis = 1)
            
            sum_up_map_prom = curr_UIS_UKPF[(curr_UIS_UKPF['Номенклатура']=='Упаковочный материал') & 
                                           (curr_UIS_UKPF['mapping']=='промпереработка')][current_month].sum()
            
            po_gp = all_zat-sum_up_map_prom
            
            curr_UIS_UKPF['расп уп мат'] = curr_UIS_UKPF.apply(lambda x: x[current_month]/po_gp*all_zat
                                           if x['Номенклатура'] == 'Упаковочный материал' and x['mapping'] !='промпереработка'
                                           else 0 ,axis = 1)
            
            curr_Production_output_UKPF['Упаковочный материал тыс. тг'] = \
            curr_Production_output_UKPF.apply(lambda x:func(x,'расп уп мат', curr_UIS_UKPF, 'Упаковочный материал') ,axis = 1)
            
            

        # ------------------------------
        
#         if ind___+1==2:
#             return curr_Production_output_UKPF

        curr_Production_output_UKPF['Упаковочный материал тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Упаковочный материал тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)
        
        
        
        
        # ------------------------------

        curr_Production_output_UKPF['Итого с/c с прямыми расходами тыс. тг'] = curr_Production_output_UKPF['C/c мясосырья, Птицеводство тыс. тг'] + \
        curr_Production_output_UKPF['Специи, добавки тыс. тг'] + \
        curr_Production_output_UKPF['Упаковочный материал тыс. тг']

        # ------------------------------

        curr_Production_output_UKPF['Итого с/c с прямыми расходами тг/кг'] = curr_Production_output_UKPF['C/c мясосырья, Птицеводство тг/кг'] + \
        curr_Production_output_UKPF['Специи, добавки тг/кг'] + \
        curr_Production_output_UKPF['Упаковочный материал тг/кг']

        # ------------------------------

        dict_slaughter_costs = rev_to_dict(mapping['Мэппинг '+key+' Затраты']['Артикул'],mapping['Мэппинг '+key+' Затраты']['Upstream'])
        curr_Production_output_UKPF['Затраты на убой и потрошение, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_slaughter_costs,'Upstream') ,axis = 1).astype('str')

        # ------------------------------

        def func(x, col, df, df1, filter_):
            sum_ = df[df[filter_] == "да"][col].sum()

            if x[filter_] == 'да':
                return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['upstream']['УиП']['всего'].sum()/1000
            else:
                return 0
            return 


        curr_Production_output_UKPF['Затраты на убой и потрошение тыс. тг'] = \
        curr_Production_output_UKPF.apply(lambda x:func(x,current_month, curr_Production_output_UKPF, NZ_UKPF,'Затраты на убой и потрошение, признак') ,axis = 1)

        # ------------------------------

        curr_Production_output_UKPF['Затраты на убой и потрошение тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на убой и потрошение тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)

        # ------------------------------

        dict_cutting_up= rev_to_dict(mapping['Мэппинг '+key+' Затраты']['Артикул'],mapping['Мэппинг '+key+' Затраты']['Разделка'])
        curr_Production_output_UKPF['Затраты на разделку, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_cutting_up,'Разделка') ,axis = 1).astype('str')

        # ------------------------------

        def func(x, col, df, df1, filter_):
            

            if x[filter_] == 'да':
                sum_ = df[df[filter_] == "да"][col].sum()
                return x[current_month]/sum_ * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['ОиР']['остальное'].sum()/1000
            elif x[filter_] == 'KFC':
                sum_ = df[df[filter_] == "KFC"][col].sum()
                return x[current_month]/sum_ * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['ОиР']['KFC'].sum()/1000  
            else:
                return 0


        curr_Production_output_UKPF['Затраты на разделку тыс. тг'] = \
        curr_Production_output_UKPF.apply(lambda x:func(x,
                                                        current_month,
                                                        curr_Production_output_UKPF,
                                                        NZ_UKPF,
                                                        'Затраты на разделку, признак'
                                                       ) ,axis = 1)
        
        

        # -------------------------------

        curr_Production_output_UKPF['Затраты на разделку тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на разделку тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)

        # -------------------------------

        dict_cooling_down= rev_to_dict(mapping['Мэппинг '+key+' Затраты']['Артикул'],mapping['Мэппинг '+key+' Затраты']['Охлаждение'])
        curr_Production_output_UKPF['Затраты на охлаждение, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_cooling_down,'Охлаждение') ,axis = 1).astype('str')

        # -------------------------------

        def func(x, col, df, df1, filter_):
            sum_ = df[df[filter_] == "да"][col].sum()

            if x[filter_] == 'да':
                return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['охлаждение']['охл'].sum()/1000
            else:
                return 0
            return 


        curr_Production_output_UKPF['Затраты на охлаждение тыс. тг'] = \
        curr_Production_output_UKPF.apply(lambda x:func(x,
                                                        current_month,
                                                        curr_Production_output_UKPF,
                                                        NZ_UKPF,
                                                        'Затраты на охлаждение, признак'
                                                       ) ,axis = 1)
        
        
        # -------------------------------

        curr_Production_output_UKPF['Затраты на охлаждение тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на охлаждение тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)

        # -------------------------------

        dict_freezing= rev_to_dict(mapping['Мэппинг '+key+' Затраты']['Артикул'],mapping['Мэппинг '+key+' Затраты']['Заморозка'])
        curr_Production_output_UKPF['Затраты на заморозку, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_freezing,'Заморозка') ,axis = 1).astype('str')

        # -------------------------------

        def func(x, col, df, df1, filter_):
            sum_ = df[df[filter_] == "да"][col].sum()

            if x[filter_] == 'да':
                return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['охлаждение']['зам'].sum()/1000
            else:
                return 0
            return 


        curr_Production_output_UKPF['Затраты на заморозку тыс. тг'] = \
        curr_Production_output_UKPF.apply(lambda x:func(x,
                                                        current_month,
                                                        curr_Production_output_UKPF,
                                                        NZ_UKPF,
                                                        'Затраты на заморозку, признак'
                                                       ) ,axis = 1)

        # --------------------------------

        curr_Production_output_UKPF['Затраты на заморозку тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на заморозку тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)

        # --------------------------------

        dict_individual_pacts= rev_to_dict(mapping['Мэппинг '+key+' Затраты']['Артикул'],mapping['Мэппинг '+key+' Затраты']['индив. пакет'])
        curr_Production_output_UKPF['Затраты на Индив. пакет, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_individual_pacts,'индив. пакет') ,axis = 1).astype('str')

        # --------------------------------

        dict_background= rev_to_dict(mapping['Мэппинг '+key+' Затраты']['Артикул'],mapping['Мэппинг '+key+' Затраты']['подложка'])
        curr_Production_output_UKPF['Затраты на Подложку, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_background,'подложка') ,axis = 1).astype('str')

        # --------------------------------

        dict_group_package= rev_to_dict(mapping['Мэппинг '+key+' Затраты']['Артикул'],mapping['Мэппинг '+key+' Затраты']['групповой пакет'])
        curr_Production_output_UKPF['Затраты на Групп. Пакет, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_group_package,'групповой пакет') ,axis = 1).astype('str')

        # --------------------------------


        NZ_UKPF = input_proc_packaging(curr_Production_output_UKPF,
                             ['Затраты на Индив. пакет, признак', \
                              'Затраты на Подложку, признак', \
                              'Затраты на Групп. Пакет, признак'],
                             current_month,
                             'Норма упаковки, кг/чел*час',
                             NZ_UKPF,
                             ['индив. пакет', 'подложка', 'групповой пакет'],
                             'Участок фасовки и упаковки '
                            )
        
        
        
        # --------------------------------

        def func(x, col, df, df1, filter_):
            sum_ = df[df[filter_] == "да"][col].sum()

            if x[filter_] == 'да':
                return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['Упак']['индив. пакет'].sum()/1000
            else:
                return 0
            return 


        curr_Production_output_UKPF['Затраты на Индив. пакет тыс. тг'] = \
        curr_Production_output_UKPF.apply(lambda x:func(x,
                                                        current_month,
                                                        curr_Production_output_UKPF,
                                                        NZ_UKPF,
                                                        'Затраты на Индив. пакет, признак'
                                                       ) ,axis = 1)

        # --------------------------------
        
        curr_Production_output_UKPF['Затраты на Индив. пакет тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на Индив. пакет тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)

        
        
        
        # --------------------------------

        def func(x, col, df, df1, filter_):
            sum_ = df[df[filter_] == "да"][col].sum()

            if x[filter_] == 'да':
                return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['Упак']['подложка'].sum()/1000
            else:
                return 0
            return 


        curr_Production_output_UKPF['Затраты на Подложку тыс. тг'] = \
        curr_Production_output_UKPF.apply(lambda x:func(x,
                                                        current_month,
                                                        curr_Production_output_UKPF,
                                                        NZ_UKPF,
                                                        'Затраты на Подложку, признак'
                                                       ) ,axis = 1)
        
#         if ind___+1==2:
#             return part_1_cons_2_per_product

        # --------------------------------

        curr_Production_output_UKPF['Затраты на Подложку тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на Подложку тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)


        # --------------------------------

        def func(x, col, df, df1, filter_):
            sum_ = df[df[filter_] == "да"][col].sum()

            if x[filter_] == 'да':
                return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['Упак']['групповой пакет'].sum()/1000
            else:
                return 0
            return 


        curr_Production_output_UKPF['Затраты на Групп. Пакет тыс. тг'] = \
        curr_Production_output_UKPF.apply(lambda x:func(x,
                                                        current_month,
                                                        curr_Production_output_UKPF,
                                                        NZ_UKPF,
                                                        'Затраты на Групп. Пакет, признак'
                                                       ) ,axis = 1)

        # --------------------------------

        curr_Production_output_UKPF['Затраты на Групп. Пакет тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на Групп. Пакет тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)

        # --------------------------------

        dict_marinate= rev_to_dict(mapping['Мэппинг '+key+' Затраты']['Артикул'],mapping['Мэппинг '+key+' Затраты']['маринация'])
        curr_Production_output_UKPF['Затраты на Маринацию, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_marinate,'маринация') ,axis = 1).astype('str')

        # --------------------------------

        def func(x, col, df, df1, filter_):
            

            if x[filter_] == 'да':
                sum_ = df[df[filter_] == "да"][col].sum()
                return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['Маринады']['остальное'].sum()/1000
            elif x[filter_] == 'KFC':
                sum_ = df[df[filter_] == "KFC"][col].sum()
                return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['Маринады']['KFC'].sum()/1000  
            else:
                return 0


        curr_Production_output_UKPF['Затраты на Маринацию тыс. тг'] = \
        curr_Production_output_UKPF.apply(lambda x:func(x,
                                                        current_month,
                                                        curr_Production_output_UKPF,
                                                        NZ_UKPF,
                                                        'Затраты на Маринацию, признак'
                                                       ) ,axis = 1)
        
        # --------------------------------

        curr_Production_output_UKPF['Затраты на Маринацию тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на Маринацию тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)

        # --------------------------------

        dict_pressing= rev_to_dict(mapping['Мэппинг '+key+' Затраты']['Артикул'],mapping['Мэппинг '+key+' Затраты']['Прессование'])
        curr_Production_output_UKPF['Затраты на Прессование признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_pressing,'Прессование') ,axis = 1).astype('str')

        # --------------------------------

        def func(x, col, df, df1, filter_):
            sum_ = df[df[filter_] == "да"][col].sum()

            if x[filter_] == 'да':
                return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['прессование']['всего'].sum()/1000
            else:
                return 0
            return 
        

        curr_Production_output_UKPF['Затраты на Прессование тыс. тг'] = \
        curr_Production_output_UKPF.apply(lambda x:func(x,
                                                        current_month,
                                                        curr_Production_output_UKPF,
                                                        NZ_UKPF,
                                                        'Затраты на Прессование признак'
                                                       ) ,axis = 1)


        # --------------------------------


        curr_Production_output_UKPF['Затраты на Прессование тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на Прессование тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)

#         if ind___+1==1:
#             return curr_Production_output_UKPF
        # --------------------------------

        curr_Production_output_UKPF['Итого накладная с/с тыс. тг'] =curr_Production_output_UKPF['Затраты на убой и потрошение тыс. тг']+ \
                                                                    curr_Production_output_UKPF['Затраты на охлаждение тыс. тг']+ \
                                                                    curr_Production_output_UKPF['Затраты на заморозку тыс. тг']+ \
                                                                    curr_Production_output_UKPF['Затраты на Индив. пакет тыс. тг']+ \
                                                                    curr_Production_output_UKPF['Затраты на Групп. Пакет тыс. тг']+ \
                                                                    curr_Production_output_UKPF['Затраты на Маринацию тыс. тг']+ \
                                                                    curr_Production_output_UKPF['Затраты на Прессование тыс. тг']+ \
                                                                    curr_Production_output_UKPF['Затраты на разделку тыс. тг']+ \
                                                                    curr_Production_output_UKPF['Затраты на Подложку тыс. тг']

        curr_Production_output_UKPF['Итого накладная с/с тг/кг'] =curr_Production_output_UKPF['Затраты на убой и потрошение тг/кг']+ \
                                                                    curr_Production_output_UKPF['Затраты на охлаждение тг/кг']+ \
                                                                    curr_Production_output_UKPF['Затраты на заморозку тг/кг']+ \
                                                                    curr_Production_output_UKPF['Затраты на Индив. пакет тг/кг']+ \
                                                                    curr_Production_output_UKPF['Затраты на Групп. Пакет тг/кг']+ \
                                                                    curr_Production_output_UKPF['Затраты на Маринацию тг/кг']+ \
                                                                    curr_Production_output_UKPF['Затраты на Прессование тг/кг']+\
                                                                    curr_Production_output_UKPF['Затраты на разделку тг/кг']+ \
                                                                    curr_Production_output_UKPF['Затраты на Подложку тг/кг']
        
        

        curr_Production_output_UKPF['Итого с/c тыс. тг'] = curr_Production_output_UKPF['Итого с/c с прямыми расходами тыс. тг'] + \
                                                            curr_Production_output_UKPF['Итого накладная с/с тыс. тг']


        curr_Production_output_UKPF['Итого с/c тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Итого с/c тыс. тг'] / \
                                         x[current_month] * 1000 ,axis = 1)
        
#         if ind___+1==1:
#             return curr_Production_output_UKPF
        
#    Записали в словарь     
        cons_pr[current_month] = curr_Production_output_UKPF
        

        # таблица с остатками 
        # для новой формы остатков
        cols_join = [x for x in curr_Production_output_UKPF.rename(columns={current_month: 'Объем кг'}).columns if x.find('тг/кг')!=-1]

        cols_join.extend(['Объем кг'])
        
        if current_month.month == 1:
            

            start_date = datetime.datetime(year_report-1, 12, 31, 0, 0)

            


            f_prt = pd.DataFrame(ost_start_new_form.index)
            # общий список из остатков , производства и продаж гп
            arr_ost = f_prt.append(curr_Production_output_UKPF[['Артикул']]).append(curr_sale_finished_products_UKPF[['Артикул']]).drop_duplicates(subset=['Артикул'])

            arr_ost = arr_ost.set_index('Артикул')
            curr_Production_output_UKPF = curr_Production_output_UKPF.set_index('Артикул')

            arr_ost = arr_ost[~arr_ost.index.duplicated(keep='first')]


            # # Берем начальные остатки
            # ost_start.columns = [x[1] for x in ost_start.columns]

            cons_ost_start = pd.concat([ost_start_new_form[cols_join],arr_ost] ,axis = 1)

            ml = pd.MultiIndex.from_tuples([ ('Остаток',start_date, x) for x in  cons_ost_start.columns])

            cons_ost_start.columns =ml
            
            path_ost_nach = ('Остаток',start_date)
            path_pr = ('Приход',current_month)
            path_r = ('Расход',current_month)
            path_ost_kon = ('Остаток',current_month)
            
            
            # приход для объединения с остатками
            production_for_join_ost = curr_Production_output_UKPF.rename(columns={current_month: 'Объем кг'})[cols_join]

            # приход рассчитан ранее , поэтому просто их joinim с остатками
            ml = pd.MultiIndex.from_tuples([ ('Приход',current_month, x) for x in  production_for_join_ost.columns])
            production_for_join_ost.columns =ml

            

            # production_for_join_ost блок с выпуском (он будет актуальным)

            it = pd.concat([cons_ost_start,production_for_join_ost] ,axis = 1)
            
            
        else:
            path_ost_nach = ('Остаток',prev_month)
            path_pr = ('Приход',current_month)
            path_r = ('Расход',current_month)
            path_ost_kon = ('Остаток',current_month)
            
            # приход для объединения с остатками
            production_for_join_ost = curr_Production_output_UKPF.rename(columns={current_month: 'Объем кг'})[cols_join]
            ml = pd.MultiIndex.from_tuples([ ('Приход',current_month, x) for x in  production_for_join_ost.columns])
            production_for_join_ost.columns =ml

            it = pd.concat([it,production_for_join_ost] ,axis = 1)
            
            it = it.set_index('Артикул')
        

        # формируем расходную часть 
        
        

        ml = pd.MultiIndex.from_tuples([ ('Расход',current_month, x[2]) for x in  production_for_join_ost.columns])
        arr_pash = arr_ost.copy()
        arr_pash = pd.DataFrame(columns=ml,index=arr_pash.index)

        it = pd.concat([it,arr_pash] ,axis = 1)
        
        
        
        it = it.reset_index()
        
        it.fillna(0, inplace=True)

        

        # -----------------------------------    

        def func(x,df,filter_,col ): return df[df['Артикул'] == x['Артикул'][0]][col].sum()
        it.loc[:,(path_r[0],path_r[1],'Объем кг')] = it.apply(lambda x:func(x,
                                    curr_sale_finished_products_UKPF,
                                    'Артикул',
                                    current_month
                                   ) ,axis = 1)  
        
        
        

        # -----------------------------------

        items = ['C/c мясосырья, Птицеводство тг/кг','Специи, добавки тг/кг','Упаковочный материал тг/кг','Затраты на убой и потрошение тг/кг',
                'Затраты на разделку тг/кг','Затраты на охлаждение тг/кг','Затраты на заморозку тг/кг','Затраты на Индив. пакет тг/кг',
                'Затраты на Подложку тг/кг','Затраты на Групп. Пакет тг/кг','Затраты на Маринацию тг/кг','Затраты на Прессование тг/кг']


        
        for item in items:
            it.loc[:,(path_r[0],path_r[1],item)] = ( it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')] * \
                                                    it.loc[:,(path_ost_nach[0],path_ost_nach[1],item)] + \
                                                    it.loc[:,(path_pr[0],path_pr[1],'Объем кг')] * \
                                                    it.loc[:,(path_pr[0],path_pr[1],item)] ) / \
                                                    (  it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')] + it.loc[:,(path_pr[0],path_pr[1],'Объем кг')]  )
        
   
        
        # -----------------------------------
        
        

        it.loc[:,(path_r[0],path_r[1],'Итого с/c с прямыми расходами тг/кг')] = it.loc[:,(path_r[0],path_r[1],'C/c мясосырья, Птицеводство тг/кг')] + \
                                                                                it.loc[:,(path_r[0],path_r[1],'Специи, добавки тг/кг')] + \
                                                                                it.loc[:,(path_r[0],path_r[1],'Упаковочный материал тг/кг')]

        # -----------------------------------

        it.loc[:,(path_r[0],path_r[1],'Итого накладная с/с тг/кг')] =  it.loc[:,(path_r[0],path_r[1],'Затраты на убой и потрошение тг/кг')] + \
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на разделку тг/кг')] + \
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на охлаждение тг/кг')] +\
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на заморозку тг/кг')] +\
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на Индив. пакет тг/кг')] +\
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на Подложку тг/кг')] +\
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на Групп. Пакет тг/кг')] +\
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на Маринацию тг/кг')] +\
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на Прессование тг/кг')]


        # -----------------------------------

        it.loc[:,(path_r[0],path_r[1],'Итого с/c тг/кг')] =  it.loc[:,(path_r[0],path_r[1],'Итого накладная с/с тг/кг')] + \
                                                                                it.loc[:,(path_r[0],path_r[1],'Итого с/c с прямыми расходами тг/кг')] 


        

        # Блок остатки на конец периода

        ml = pd.MultiIndex.from_tuples([ ('Остаток',current_month, x[2]) for x in  production_for_join_ost.columns])
        arr_ost_kon = arr_ost.copy()
        arr_ost_kon = pd.DataFrame(columns=ml,index=arr_ost_kon.index)
        it = it.set_index('Артикул')
        it = pd.concat([it,arr_ost_kon] ,axis = 1)
        it = it.reset_index()


        # -----------------------------------    

        def func(x,df,filter_,col ): return df[df['Артикул'] == x['Артикул'][0]][col].sum()
        it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Объем кг')] = it.apply(lambda x:func(x,
                                    curr_balance_depot_stock_UKPF,
                                    'Артикул',
                                    current_month
                                   ) ,axis = 1)  

        # -----------------------------------

        
        for item in items:
            it.loc[:,(path_ost_kon[0],path_ost_kon[1],item)] = ( it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')] * \
                                                    it.loc[:,(path_ost_nach[0],path_ost_nach[1],item)] + \
                                                    it.loc[:,(path_pr[0],path_pr[1],'Объем кг')] * \
                                                    it.loc[:,(path_pr[0],path_pr[1],item)] - \
                                                    it.loc[:,(path_r[0],path_r[1],'Объем кг')] * \
                                                    it.loc[:,(path_r[0],path_r[1],item)] ) / \
                                                    (  it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')] + it.loc[:,(path_pr[0],path_pr[1],'Объем кг')] - it.loc[:,(path_r[0],path_r[1],'Объем кг')])


        # -----------------------------------

        it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Итого с/c с прямыми расходами тг/кг')] = it.loc[:,(path_ost_kon[0],path_ost_kon[1],'C/c мясосырья, Птицеводство тг/кг')] + \
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Специи, добавки тг/кг')] + \
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Упаковочный материал тг/кг')]   


        # -----------------------------------

        it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Итого накладная с/с тг/кг')] =  it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на убой и потрошение тг/кг')] + \
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на разделку тг/кг')] + \
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на охлаждение тг/кг')] +\
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на заморозку тг/кг')] +\
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на Индив. пакет тг/кг')] +\
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на Подложку тг/кг')] +\
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на Групп. Пакет тг/кг')] +\
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на Маринацию тг/кг')] +\
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на Прессование тг/кг')]
        
        #------------------------------------
        
        
        it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Итого с/c тг/кг')] =  it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Итого накладная с/с тг/кг')] + \
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Итого с/c с прямыми расходами тг/кг')] 
        
        
        if ind___+1==mon:
            return cons_pr, it, per_1,per_2
        
    return cons_pr, it
    
    

In [ ]:
year_report = 2021
month = 6
a = get_ssmp_ukpf(directory,
              'МПФ.xlsx',
              'Mapping.xlsx',
              'Коэффициент ценности.xlsx',
              'Остатки на начало года МПФ.xlsx',
                month,
                global_index
             )

In [ ]:
year_report = 2021
month = 6
a = get_ssmp_ukpf(directory,
              'УКПФ.xlsx',
              'Mapping.xlsx',
              'Коэффициент ценности.xlsx',
              'Остатки на начало года УКПФ.xlsx',
                month,
                global_index
             )

In [ ]:
a[a['Артикул']==411]

In [ ]:
a['усл.ед.'].sum()

In [ ]:
a['%%от тушки'].sum()

In [ ]:
a['Доля стоимости 2 передела'].sum()

In [ ]:
a

In [ ]:
294.917823/633.43

In [ ]:
294.987823/633.43

In [ ]:
a[a['Артикул']==1066]

In [ ]:
year_report = 2021
month = 6
global_index = ['Артикул', 'Продукция', 'Номенклатура', 'Канал', 'Тип']
directory = 'C:\\Users\\Кедрун Никита\\PycharmProjects\\akz_\\input\\'

prod_UKPF,ost_UKPF,per_1_ukpf,per_2_ukpf = get_ssmp_ukpf(directory,
              'УКПФ.xlsx',
              'Mapping.xlsx',
              'Коэффициент ценности.xlsx',
              'Остатки на начало года УКПФ.xlsx',
                month,
                global_index
             )

prod_MPF,ost_MPF,per_1_mpf,per_2_mpf = get_ssmp_ukpf(directory,
              'МПФ.xlsx',
              'Mapping.xlsx',
              'Коэффициент ценности.xlsx',
              'Остатки на начало года МПФ.xlsx',
                month,
                global_index
             )

In [ ]:
def get_amd_sebes(prod_UKPF,ost_UKPF,prod_MPF,ost_MPF,year_report,ost_nach_g,directory,filename,month,global_index):
    
    
    ost_start_new_dip =  pd.read_excel(directory+ost_nach_g,sheet_name='Sheet1',index_col=0)
    
    budj =  pd.read_excel(directory+filename,sheet_name='Продажи бюдж',header=[0,1])
    budj.rename(columns=dict((x[1],'') for x in budj.columns if x[1].find('Unnamed')!=-1) ,inplace=True)
    budj = budj.dropna(axis='index', how='any', subset=[('Артикул','','')])
    budj  = retype_multiindex(budj, 'int', 'Артикул', 0)
    budj = budj.set_index('Артикул')
    
    ost_AMD =  pd.read_excel(directory+filename,sheet_name='Остатки АМД',header=[0,1])
    ost_AMD.rename(columns=dict((x[1],'') for x in ost_AMD.columns if x[1].find('Unnamed')!=-1) ,inplace=True)
    ost_AMD = ost_AMD.dropna(axis='index', how='any', subset=[('Артикул','','')])
    ost_AMD  = retype_multiindex(ost_AMD, 'int', 'Артикул', 0)
    ost_AMD = ost_AMD.set_index('Артикул')
    
    
    
    ost_AMP =  pd.read_excel(directory+filename,sheet_name='Остатки АМП')
#     ost_AMP.rename(columns=dict((x[1],'') for x in ost_AMD.columns if x[1].find('Unnamed')!=-1) ,inplace=True)
    ost_AMP = ost_AMP.dropna(axis='index', how='any', subset=['Артикул'])
    ost_AMP  = retype_index(ost_AMP, 'int', 'Артикул')
    ost_AMP = ost_AMP.set_index('Артикул')
    
    
    sebes_amp =  pd.read_excel(directory+filename,sheet_name='Для СС АМП',header=[0,1])
    sebes_amp.rename(columns=dict((x[1],'') for x in sebes_amp.columns if x[1].find('Unnamed')!=-1) ,inplace=True)
    sebes_amp = sebes_amp.dropna(axis='index', how='any', subset=[('Артикул','','')])
    sebes_amp  = retype_multiindex(sebes_amp, 'int', 'Артикул', 0)
    sebes_amp = sebes_amp.set_index('Артикул')
    
    
    
    iter_months = [datetime.datetime(year_report,x,calendar.monthrange(year_report,x)[1],0,0) for x in range(1,13)]
    
    ost_start = ost_UKPF.copy()
    
#     ost_UKPF = ost_UKPF.set_index('Артикул')
#     ost_MPF = ost_MPF.set_index('Артикул')
    
    for ind___, current_month in enumerate(iter_months):
        print(current_month)
        
#         Текущие данные по бюджету продаж
        
        curr_budj = get_current_data(budj, global_index, current_month)
        curr_ost_AMD = get_current_data(ost_AMD, global_index, current_month)
        
#         нужна для формирования таблицы с остатками
        if ind___>0:
            prev_month = iter_months[ind___-1]
            
            
            
#         bolv = pd.DataFrame()
        cols_join = []    
            
        if current_month.month == 1:
            
            start_date = datetime.datetime(year_report-1, 12, 31, 0, 0)
            
            cols_join = [x for x in ost_UKPF['Остаток'][start_date].rename(columns={current_month: 'Объем кг'}).columns if x.find('тг/кг')!=-1]
            
            cons_ost_start = pd.DataFrame(index = ost_start_new_dip.index,data = ost_start_new_dip)
            bolv = cons_ost_start.copy()
            for col in bolv.columns:bolv[col] = 0
#             f_ost_nach = pd.DataFrame(data = ost_UKPF['Остаток'][start_date],index = ost_UKPF.loc[:,'Артикул'])
            
#             f_ost_nach.index.name = 'Артикул'


            
            ml = pd.MultiIndex.from_tuples([ ('Остаток',start_date, x) for x in  cons_ost_start.columns])

            cons_ost_start.columns =ml
            
            path_ost_nach = ('Остаток',start_date)
            path_pr = ('Приход',current_month)
            path_r = ('Расход',current_month)
            path_ost_kon = ('Остаток',current_month)
            
            ost_UKPF  =ost_UKPF.set_index('Артикул')
            ost_MPF = ost_MPF.set_index('Артикул')

        else:
            
            path_ost_nach = ('Остаток',prev_month)
            path_pr = ('Приход',current_month)
            path_r = ('Расход',current_month)
            path_ost_kon = ('Остаток',current_month)
            
            
            
#         Обрботка приходной части
        
        prih = bolv.copy()
#         prih = prih.reset_index()
        
        
        ost_UKPF_ = ost_UKPF['Расход'][current_month].copy()
        ost_MPF_ = ost_MPF['Расход'][current_month].copy()
        ost_UKPF_ = ost_UKPF_.reset_index()
        ost_MPF_ = ost_MPF_.reset_index()
        
        
        
        ml = pd.MultiIndex.from_tuples([ ('Приход',current_month, x) for x in  prih.columns])

        prih.columns =ml
        
        if current_month.month == 1:
            it = pd.concat([cons_ost_start,prih] ,axis = 1)
            
        else:
            it = it.set_index('Артикул')
            it = pd.concat([it,prih] ,axis = 1)
        
        
#         ------------------------------------------------------------------------------------------------------------

        
        
        it = it.reset_index()
        
        def func(x,df,df1,filter_): 
            ukpf = df[df[filter_] == x[filter_][0]]['Объем кг'].sum()
            mpf = df1[df1[filter_] == x[filter_][0]]['Объем кг'].sum()
            return ukpf+mpf

        it.loc[:,(path_pr[0],path_pr[1],'Объем кг')] = it.apply(lambda x:func(x,
                                    ost_UKPF_,
                                    ost_MPF_,              
                                    'Артикул'
                                   ) ,axis = 1)  
        
        # --------------------------------
        
        

        def func(x,df,df1,filter_,col3 ): 
            return ( df[df[filter_] == x[filter_][0]]['Объем кг'].sum() * \
                    df[df[filter_] == x[filter_][0]][col3].sum() + \
                    df1[df1[filter_] == x[filter_][0]]['Объем кг'].sum() * \
                    df1[df1[filter_] == x[filter_][0]][col3].sum() ) / x['Приход'][current_month]['Объем кг']
        
        
        items = ['C/c мясосырья, Птицеводство тг/кг','Специи, добавки тг/кг','Упаковочный материал тг/кг','Затраты на убой и потрошение тг/кг',
                'Затраты на разделку тг/кг','Затраты на охлаждение тг/кг','Затраты на заморозку тг/кг','Затраты на Индив. пакет тг/кг',
                'Затраты на Подложку тг/кг','Затраты на Групп. Пакет тг/кг','Затраты на Маринацию тг/кг','Затраты на Прессование тг/кг']


        for item in items:

            it.loc[:,(path_pr[0],path_pr[1],item)] = it.apply(lambda x:func(x,
                                        ost_UKPF_,
                                        ost_MPF_,              
                                        'Артикул',
                                        item
                                       ) ,axis = 1) 
        
        
            
         # -----------------------------------
        
        

        it.loc[:,(path_pr[0],path_pr[1],'Итого с/c с прямыми расходами тг/кг')] = it.loc[:,(path_pr[0],path_pr[1],'C/c мясосырья, Птицеводство тг/кг')] + \
                                                                                it.loc[:,(path_pr[0],path_pr[1],'Специи, добавки тг/кг')] + \
                                                                                it.loc[:,(path_pr[0],path_pr[1],'Упаковочный материал тг/кг')]

        # -----------------------------------

        it.loc[:,(path_pr[0],path_pr[1],'Итого накладная с/с тг/кг')] =  it.loc[:,(path_pr[0],path_pr[1],'Затраты на убой и потрошение тг/кг')] + \
                                                                                it.loc[:,(path_pr[0],path_pr[1],'Затраты на разделку тг/кг')] + \
                                                                                it.loc[:,(path_pr[0],path_pr[1],'Затраты на охлаждение тг/кг')] +\
                                                                                it.loc[:,(path_pr[0],path_pr[1],'Затраты на заморозку тг/кг')] +\
                                                                                it.loc[:,(path_pr[0],path_pr[1],'Затраты на Индив. пакет тг/кг')] +\
                                                                                it.loc[:,(path_pr[0],path_pr[1],'Затраты на Подложку тг/кг')] +\
                                                                                it.loc[:,(path_pr[0],path_pr[1],'Затраты на Групп. Пакет тг/кг')] +\
                                                                                it.loc[:,(path_pr[0],path_pr[1],'Затраты на Маринацию тг/кг')] +\
                                                                                it.loc[:,(path_pr[0],path_pr[1],'Затраты на Прессование тг/кг')]
        
        
        # -----------------------------------

        it.loc[:,(path_pr[0],path_pr[1],'Итого с/c тг/кг')] =  it.loc[:,(path_pr[0],path_pr[1],'Итого накладная с/с тг/кг')] + \
                                                                                it.loc[:,(path_pr[0],path_pr[1],'Итого с/c с прямыми расходами тг/кг')] 
        
             
        
#         Заполняем расходную часть


        rash = bolv.copy()
#         rash = rash.set_index('Артикул')
        ml = pd.MultiIndex.from_tuples([ ('Расход',current_month, x) for x in  rash.columns])
        rash.columns =ml
    
#         rash = rash.reset_index()
        
        it = it.set_index('Артикул')
        it = pd.concat([it,rash] ,axis = 1)
        
        
        it = it.reset_index()
        

        
        curr_budj = curr_budj.reset_index()
        
#-----------------------
        
        def func(x,df,filter_,current_month): 
            return df[df[filter_]==x[filter_][0]][current_month]['Кол-во'].sum()

        it.loc[:,(path_r[0],path_r[1],'Объем кг')] = it.apply(lambda x:func(x,
                                    curr_budj,              
                                    'Артикул',
                                    current_month
                                   ) ,axis = 1) 
        
        
        for item in items:
            it.loc[:,(path_r[0],path_r[1],item)] = ( it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')] * \
                                                    it.loc[:,(path_ost_nach[0],path_ost_nach[1],item)] + \
                                                    it.loc[:,(path_pr[0],path_pr[1],'Объем кг')] * \
                                                    it.loc[:,(path_pr[0],path_pr[1],item)] ) / \
                                                    (  it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')] + it.loc[:,(path_pr[0],path_pr[1],'Объем кг')]  )
            
            
        
         # -----------------------------------
        
        

        it.loc[:,(path_r[0],path_r[1],'Итого с/c с прямыми расходами тг/кг')] = it.loc[:,(path_r[0],path_r[1],'C/c мясосырья, Птицеводство тг/кг')] + \
                                                                                it.loc[:,(path_r[0],path_r[1],'Специи, добавки тг/кг')] + \
                                                                                it.loc[:,(path_r[0],path_r[1],'Упаковочный материал тг/кг')]

        # -----------------------------------

        it.loc[:,(path_r[0],path_r[1],'Итого накладная с/с тг/кг')] =  it.loc[:,(path_r[0],path_r[1],'Затраты на убой и потрошение тг/кг')] + \
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на разделку тг/кг')] + \
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на охлаждение тг/кг')] +\
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на заморозку тг/кг')] +\
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на Индив. пакет тг/кг')] +\
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на Подложку тг/кг')] +\
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на Групп. Пакет тг/кг')] +\
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на Маринацию тг/кг')] +\
                                                                                it.loc[:,(path_r[0],path_r[1],'Затраты на Прессование тг/кг')]
        
        
        # -----------------------------------

        it.loc[:,(path_r[0],path_r[1],'Итого с/c тг/кг')] =  it.loc[:,(path_r[0],path_r[1],'Итого накладная с/с тг/кг')] + \
                                                                                it.loc[:,(path_r[0],path_r[1],'Итого с/c с прямыми расходами тг/кг')] 
        
        
        
#         Заполняю блок остатки на конец периода

        
        ost_con = bolv.copy()
#         rash = rash.set_index('Артикул')
        ml = pd.MultiIndex.from_tuples([ ('Остаток',current_month, x) for x in  ost_con.columns])
        ost_con.columns =ml
        it = it.set_index('Артикул')
        it = pd.concat([it,ost_con] ,axis = 1)
        
        
        
        it = it.reset_index()
        
        curr_ost_AMD = curr_ost_AMD.reset_index()
        
#         ------------------------------------
        
        def func(x,df,filter_,current_month): 
            return df[df[filter_]==x[filter_][0]][current_month]['остаток'].sum()

        it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Объем кг')] = it.apply(lambda x:func(x,
                                    curr_ost_AMD,              
                                    'Артикул',
                                    current_month
                                   ) ,axis = 1) 
        
        
        for item in items:
            it.loc[:,(path_ost_kon[0],path_ost_kon[1],item)] = ( it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')] * \
                                                    it.loc[:,(path_ost_nach[0],path_ost_nach[1],item)] + \
                                                    it.loc[:,(path_pr[0],path_pr[1],'Объем кг')] * \
                                                    it.loc[:,(path_pr[0],path_pr[1],item)] - \
                                                    it.loc[:,(path_r[0],path_r[1],'Объем кг')] * \
                                                    it.loc[:,(path_r[0],path_r[1],item)] ) / \
                                                    (  it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')] + it.loc[:,(path_pr[0],path_pr[1],'Объем кг')] - it.loc[:,(path_r[0],path_r[1],'Объем кг')])


        # -----------------------------------

        it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Итого с/c с прямыми расходами тг/кг')] = it.loc[:,(path_ost_kon[0],path_ost_kon[1],'C/c мясосырья, Птицеводство тг/кг')] + \
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Специи, добавки тг/кг')] + \
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Упаковочный материал тг/кг')]   


        # -----------------------------------

        it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Итого накладная с/с тг/кг')] =  it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на убой и потрошение тг/кг')] + \
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на разделку тг/кг')] + \
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на охлаждение тг/кг')] +\
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на заморозку тг/кг')] +\
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на Индив. пакет тг/кг')] +\
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на Подложку тг/кг')] +\
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на Групп. Пакет тг/кг')] +\
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на Маринацию тг/кг')] +\
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Затраты на Прессование тг/кг')]
        
        #------------------------------------
        
        
        it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Итого с/c тг/кг')] =  it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Итого накладная с/с тг/кг')] + \
                                                                                it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Итого с/c с прямыми расходами тг/кг')] 
        
        
        if ind___+1==month:
            return it, budj, bolv, sebes_amp,ost_AMP
#         return it

In [ ]:
# на выходе статки AMD, Бюджет АМД, болванка для присодинения данныч с артикулом и всеми необходимыми мтолбцами блока, и таблица 'Для СС АМП'

ost_AMD, budj_AMD, bolv, sebes_amp, ost_AMP = get_amd_sebes(prod_UKPF,
              ost_UKPF,
              prod_MPF,
              ost_MPF,
              year_report,
             'Остатки на начало года АМД.xlsx',
             directory,
             'АМП и АМД.xlsx',
             month,
            global_index)

In [ ]:
def get_ss_amp(month,budj_AMD,global_index,year_report,bolv1,ost_AMD,sebes_amp,mapping,amort,ost_nach_g,ost_AMP):

#     эта структура хранит верхнюю таблицу пг_сс_амп формат: дата - 'выход_AMD' и дата - Списано в производство АМП
    upper_table = {}
    sebes_real_pr_AMD = {}
    
    ost_start_new_amp =  pd.read_excel(directory+ost_nach_g,sheet_name='Sheet1',index_col=0)
    
    mapping =  pd.read_excel(directory+mapping,sheet_name='Mapping',header=[0,1] )
    mapping  = retype_multiindex(mapping, 'int', 'Артикул', 1)
    index_for_amp_tables = dict_keys_free('Мэппинг АМП',mapping,['Артикул','Наименование АМД','Тип'])
    

    iter_months = [datetime.datetime(year_report,x,calendar.monthrange(year_report,x)[1],0,0) for x in range(1,13)]
    ost_AMD = ost_AMD.set_index('Артикул')
    
    
    amort =  pd.read_excel(directory+amort,sheet_name='Амортизация')
    
    for ind___, current_month in enumerate(iter_months):
        print(current_month)
        
        curr_amort = get_current_data(amort, global_index, current_month)
        
        
        
        curr_sebes_amp = get_current_data(sebes_amp, global_index, current_month)[current_month].reset_index()
        curr_budj = get_current_data(budj_AMD.reset_index().set_index(['Артикул','канал']), global_index, current_month)[current_month].reset_index()
        curr_ost_AMP = get_current_data(ost_AMP, global_index, current_month)
        output_amd = bolv1.copy().reset_index()
        
        
        
#         собираю блок закуп у амд
        
        def func(x,df,filter_,filter__): 
            return df[(df[filter_] == x[filter_]) & (df[filter__] == 'АМП пром')]['Кол-во'].sum()

        output_amd['Объем кг'] = output_amd.apply(lambda x:func(x,
                                    curr_budj,             
                                    'Артикул',
                                    'канал'
                                   ) ,axis = 1)  
        
        
        
        # --------------------------------
        
        current_rash = ost_AMD['Расход'][current_month].copy().reset_index()
        
        
        items = ['C/c мясосырья, Птицеводство тг/кг','Специи, добавки тг/кг','Упаковочный материал тг/кг','Затраты на убой и потрошение тг/кг',
                'Затраты на разделку тг/кг','Затраты на охлаждение тг/кг','Затраты на заморозку тг/кг','Затраты на Индив. пакет тг/кг',
                'Затраты на Подложку тг/кг','Затраты на Групп. Пакет тг/кг','Затраты на Маринацию тг/кг','Затраты на Прессование тг/кг']
        
        
        for item in items:
            
            def func(x,df,filter_,filter__): 
                return df[df[filter_] == x[filter_]][item].sum()

            output_amd[item] = output_amd.apply(lambda x:func(x,
                                        current_rash,             
                                        'Артикул',
                                        item
                                       ) ,axis = 1)
            
            
            output_amd[item] = (output_amd['Объем кг']*output_amd[item])/output_amd['Объем кг']
            
            
        output_amd['Итого с/c с прямыми расходами тг/кг'] = output_amd['C/c мясосырья, Птицеводство тг/кг'] + \
                                                            output_amd['Специи, добавки тг/кг'] + \
                                                            output_amd['Упаковочный материал тг/кг']   

        # -----------------------------------

        output_amd['Итого накладная с/с тг/кг'] =  output_amd['Затраты на убой и потрошение тг/кг'] + \
                                                        output_amd['Затраты на разделку тг/кг'] + \
                                                        output_amd['Затраты на охлаждение тг/кг'] +\
                                                        output_amd['Затраты на заморозку тг/кг'] +\
                                                        output_amd['Затраты на Индив. пакет тг/кг'] +\
                                                        output_amd['Затраты на Подложку тг/кг'] +\
                                                        output_amd['Затраты на Групп. Пакет тг/кг'] +\
                                                        output_amd['Затраты на Маринацию тг/кг'] +\
                                                        output_amd['Затраты на Прессование тг/кг']
        
        #------------------------------------
        
        output_amd['Итого с/c тг/кг'] =  output_amd['Итого накладная с/с тг/кг'] + \
                                        output_amd['Итого с/c с прямыми расходами тг/кг'] 
        
        
        
        
#         Запись в словарь блока закуп у АМД
        upper_table[current_month] = output_amd
        
        
#         Зполняю себестоимость реализованной продукции АМП

        sebes_prod_amp = pd.DataFrame(index = index_for_amp_tables['Артикул'],columns = output_amd.set_index('Артикул').columns)
        sebes_prod_amp = sebes_prod_amp.reset_index()
        sebes_prod_amp = sebes_prod_amp.drop_duplicates(subset=['Артикул'])
        sebes_prod_amp = sebes_prod_amp[sebes_prod_amp['Артикул']!=0]
        
        def func(x,df,filter_,col): 
            return df[df[filter_] == x[filter_]][col].sum()

        sebes_prod_amp['Объем кг'] = sebes_prod_amp.apply(lambda x:func(x,
                                    curr_sebes_amp,             
                                    'Артикул',
                                    'Объем'
                                   ) ,axis = 1)
        
        
        
        sebes_prod_amp['Произв-я с/с по 1С, с амортизацией тыс. тг'] = sebes_prod_amp.apply(lambda x:func(x,
                                    curr_sebes_amp,             
                                    'Артикул',
                                    'Произв с/с с аморт'
                                   ) ,axis = 1) / 1000
        
        
        
        sebes_prod_amp['Сырьевая с/с по 1С тыс. тг'] = sebes_prod_amp.apply(lambda x:func(x,
                                    curr_sebes_amp,             
                                    'Артикул',
                                    'Сырьевая с/с'
                                   ) ,axis = 1) / 1000
        
        
        sebes_prod_amp['С/с мясосырья с наценкой по 1С тыс. тг'] = sebes_prod_amp.apply(lambda x:func(x,
                                    curr_sebes_amp,             
                                    'Артикул',
                                    'С\с мясо'
                                   ) ,axis = 1) / 1000
        
       
        def func(x,sebes_prod_amp,curr_amort): 
            return x['Произв-я с/с по 1С, с амортизацией тыс. тг']/sebes_prod_amp['Произв-я с/с по 1С, с амортизацией тыс. тг'].sum() * \
                    ( sebes_prod_amp['Произв-я с/с по 1С, с амортизацией тыс. тг'].sum() - curr_amort[current_month]/1000 )

        sebes_prod_amp['Произв-я с/с по 1С, без амортизации тыс. тг'] = sebes_prod_amp.apply(lambda x:func(x,
                                    sebes_prod_amp,
                                    curr_amort
                                   ) ,axis = 1)
        
        
        def func(x,sebes_prod_amp,output_amd): 
            return x['С/с мясосырья с наценкой по 1С тыс. тг']/sebes_prod_amp['С/с мясосырья с наценкой по 1С тыс. тг'].sum() * \
                    output_amd['C/c мясосырья, Птицеводство тг/кг'].sum()

        sebes_prod_amp['C/c мясосырья, Птицеводство тыс. тг'] = sebes_prod_amp.apply(lambda x:func(x,
                                    sebes_prod_amp,
                                    output_amd
                                   ) ,axis = 1)
        
        
        def func(x,sebes_prod_amp,output_amd,df,filter_,col): 
            return x['Объем кг'] / sebes_prod_amp['Объем кг'].sum() * sebes_prod_amp['Специи, добавки тг/кг'].sum() + \
            df[df[filter_] == x[filter_]][col].sum() /1000

        sebes_prod_amp['Специи, добавки тыс. тг'] = sebes_prod_amp.apply(lambda x:func(x,
                                    sebes_prod_amp,
                                    output_amd,
                                    curr_sebes_amp,
                                    'Артикул',
                                    'С\с специи'
                                   ) ,axis = 1)
        
        
        def func(x,sebes_prod_amp,output_amd,df,filter_,col): 
            return x['Объем кг'] / sebes_prod_amp['Объем кг'].sum() * sebes_prod_amp['Упаковочный материал тг/кг'].sum() + \
            df[df[filter_] == x[filter_]][col].sum() /1000

        sebes_prod_amp['Упаковочный материал тыс. тг'] = sebes_prod_amp.apply(lambda x:func(x,
                                    sebes_prod_amp,
                                    output_amd,
                                    curr_sebes_amp,
                                    'Артикул',
                                    'С\с упаковка'
                                   ) ,axis = 1)
        
        dict_type_prod_amp = rev_to_dict(index_for_amp_tables['Артикул'],index_for_amp_tables['Тип'])
        sebes_prod_amp['Тип продукции'] = sebes_prod_amp.apply(lambda x: insert_vpr(x['Артикул'],dict_type_prod_amp,'Тип') ,axis = 1).astype('str')
        
        
        
        def func(x,output_amd,filter_,col,col___): 
            if x['Тип продукции']=='ГП':
                return x[col]/ sebes_prod_amp[sebes_prod_amp[filter_] == 'ГП'][col].sum() * \
                    output_amd[col___+' тг/кг'].sum()
            else:
                return 0
            
        cols = ['Затраты на убой и потрошение','Затраты на разделку','Затраты на охлаждение','Затраты на заморозку',
               'Затраты на Индив. пакет','Затраты на Подложку','Затраты на Групп. Пакет','Затраты на Маринацию','Затраты на Прессование']
        
        for col___ in cols:
            
            sebes_prod_amp[col___+' тыс. тг'] = sebes_prod_amp.apply(lambda x:func(x,
                                        output_amd,
                                        'Тип продукции',
                                        'Объем кг',
                                        col___
                                       ) ,axis = 1)
           
        sebes_prod_amp['Прозв-е затраты АМП тыс. тг'] = sebes_prod_amp['Произв-я с/с по 1С, без амортизации тыс. тг'] - sebes_prod_amp['Сырьевая с/с по 1С тыс. тг']
        
        
        items = ['C/c мясосырья, Птицеводство','Специи, добавки','Упаковочный материал','Затраты на убой и потрошение',
                'Затраты на разделку','Затраты на охлаждение','Затраты на заморозку','Затраты на Индив. пакет',
                'Затраты на Подложку','Затраты на Групп. Пакет','Затраты на Маринацию','Затраты на Прессование','Прозв-е затраты АМП']
        
        
        for item in items:
            sebes_prod_amp[item+' тг/кг'] = sebes_prod_amp[item+' тыс. тг']  / sebes_prod_amp['Объем кг'] * 1000
            
#         Записываю  Себестоимость реализованной продукции АМП в словарь
        sebes_real_pr_AMD[current_month] = sebes_prod_amp
    
            
#         закончил формрование Себестоимость реализованной продукции АМП

#        начинаю формирование Себестоимость реализованной продукции АМП с учетом остатков

        
        
        if ind___>0:
            prev_month = iter_months[ind___-1]
            
            
            
            
        if current_month.month == 1:
            cols_join = []
            
            cols_join = [x for x in sebes_prod_amp.columns if x.find('тг/кг')!=-1]
            cols_join.extend(['Объем кг'])
            
            start_date = datetime.datetime(year_report-1, 12, 31, 0, 0)
            
            
            cons_ost_start = pd.DataFrame(index = ost_start_new_amp.index,data = ost_start_new_amp)
            bolv = cons_ost_start.copy()
            
            for col in bolv.columns:bolv[col] = 0
#             f_ost_nach = pd.DataFrame(data = ost_UKPF['Остаток'][start_date],index = ost_UKPF.loc[:,'Артикул'])
            
#             f_ost_nach.index.name = 'Артикул'


            
            ml = pd.MultiIndex.from_tuples([ ('Остаток',start_date, x) for x in  cons_ost_start.columns])

            cons_ost_start.columns =ml
            
            path_ost_nach = ('Остаток',start_date)
            path_pr = ('Приход',current_month)
            path_r = ('Расход',current_month)
            path_ost_kon = ('Остаток',current_month)
            

        else:
            
            path_ost_nach = ('Остаток',prev_month)
            path_pr = ('Приход',current_month)
            path_r = ('Расход',current_month)
            path_ost_kon = ('Остаток',current_month)
            
            
        
        
    
#         Обрботка приходной части
        
#         prih = bolv.copy()
        
        
#         prih = prih.reset_index()
        
        
        
        prih = sebes_prod_amp.set_index('Артикул')[cols_join]
        
        
        ml = pd.MultiIndex.from_tuples([ ('Приход',current_month, x) for x in  prih.columns])

        prih.columns =ml
        
        
        
        if current_month.month == 1:
            it = pd.concat([cons_ost_start,prih] ,axis = 1)
            
            
        else:
            it = it.set_index('Артикул')
            it = pd.concat([it,prih] ,axis = 1)
            
        
        
        
        
#         ------------------------------------------------------------------------------------------------------------

        
        
        it = it.reset_index()
        
        cols_sum = ['C/c мясосырья, Птицеводство тг/кг','Специи, добавки тг/кг','Упаковочный материал тг/кг','Затраты на убой и потрошение тг/кг',
                   'Затраты на разделку тг/кг','Затраты на охлаждение тг/кг','Затраты на заморозку тг/кг','Затраты на Индив. пакет тг/кг',
                   'Затраты на Подложку тг/кг','Затраты на Групп. Пакет тг/кг','Затраты на Маринацию тг/кг','Затраты на Прессование тг/кг',
                   'Прозв-е затраты АМП тг/кг']

        it.loc[:,(path_pr[0],path_pr[1],'Итого с/c тг/кг')] =  it.loc[:,(path_pr[0],path_pr[1],cols_sum)].sum(axis = 1)
                                                            
        
        
#         Заполняем расходную часть


        rash = bolv.copy()
#         rash = rash.set_index('Артикул')
        ml = pd.MultiIndex.from_tuples([ ('Расход',current_month, x) for x in  rash.columns])
        rash.columns =ml
    
#         rash = rash.reset_index()
        
        it = it.set_index('Артикул')
        it = pd.concat([it,rash] ,axis = 1)
        
        
        it = it.reset_index()
        
        
        
        curr_budj = curr_budj.reset_index()
        
#-----------------------
        
        
    
        def func(x,df,filter_,current_month): 
            return df[( df[filter_]==x[filter_][0] ) & (df['канал']=='ОПТ АМП')]['Кол-во'].sum()

        it.loc[:,(path_r[0],path_r[1],'Объем кг')] = it.apply(lambda x:func(x,
                                    curr_budj,              
                                    'Артикул',
                                    current_month
                                   ) ,axis = 1) 
        
        items = ['C/c мясосырья, Птицеводство тг/кг','Специи, добавки тг/кг','Упаковочный материал тг/кг','Затраты на убой и потрошение тг/кг',
                'Затраты на разделку тг/кг','Затраты на охлаждение тг/кг','Затраты на заморозку тг/кг','Затраты на Индив. пакет тг/кг',
                'Затраты на Подложку тг/кг','Затраты на Групп. Пакет тг/кг','Затраты на Маринацию тг/кг','Затраты на Прессование тг/кг','Прозв-е затраты АМП тг/кг']
        
        for item in items:
            it.loc[:,(path_r[0],path_r[1],item)] = ( it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')] * \
                                                    it.loc[:,(path_ost_nach[0],path_ost_nach[1],item)] + \
                                                    it.loc[:,(path_pr[0],path_pr[1],'Объем кг')] * \
                                                    it.loc[:,(path_pr[0],path_pr[1],item)] ) / \
                                                    (  it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')] + it.loc[:,(path_pr[0],path_pr[1],'Объем кг')]  )
            
          
        
        
        # -----------------------------------

        it.loc[:,(path_r[0],path_r[1],'Итого с/c тг/кг')] =  it.loc[:,(path_r[0],path_r[1],cols_sum)].sum(axis = 1) 
        
        
        
        
#         Заполняю блок остатки на конец периода

        
        ost_con = bolv.copy()
#         rash = rash.set_index('Артикул')
        ml = pd.MultiIndex.from_tuples([ ('Остаток',current_month, x) for x in  ost_con.columns])
        ost_con.columns =ml
        it = it.set_index('Артикул')
        it = pd.concat([it,ost_con] ,axis = 1)
        
        
        
        it = it.reset_index()
        
        
        
        curr_ost_AMP = curr_ost_AMP.reset_index()
        
        
        
#         ------------------------------------
        
        def func(x,df,filter_,current_month): 
            return df[df[filter_]==x[filter_][0]][current_month].sum()

        it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Объем кг')] = it.apply(lambda x:func(x,
                                    curr_ost_AMP,              
                                    'Артикул',
                                    current_month
                                   ) ,axis = 1) 
         
        
        for item in items:
            
            it.loc[:,(path_ost_kon[0],path_ost_kon[1],item)] = it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')]  - \
                                                                it.loc[:,(path_pr[0],path_pr[1],'Объем кг')] 
            
            def func(x): 
                if (x[path_pr[0]][path_pr[1]]['Объем кг'] - x[path_r[0]][path_r[1]]['Объем кг']) == 0:
                    return x[path_r[0]][path_r[1]][item]
                else:
                    
                    return ( x[path_ost_nach[0]][path_ost_nach[1]]['Объем кг'] * x[path_ost_nach[0]][path_ost_nach[1]][item] + \
                            x[path_pr[0]][path_pr[1]]['Объем кг'] * x[path_pr[0]][path_pr[1]][item] + \
                            x[path_r[0]][path_r[1]]['Объем кг'] * x[path_r[0]][path_r[1]][item])  / \
                            ( x[path_ost_nach[0]][path_ost_nach[1]]['Объем кг'] + x[path_pr[0]][path_pr[1]]['Объем кг'] + \
                            x[path_r[0]][path_r[1]]['Объем кг'] )
                    
            
            it.loc[:,(path_ost_kon[0],path_ost_kon[1],item)] = it.apply(lambda x:func(x) ,axis = 1) 
            
            
            

            
            
#             it.loc[:,(path_ost_kon[0],path_ost_kon[1],item)] = ( it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')] * \
#                                                     it.loc[:,(path_ost_nach[0],path_ost_nach[1],item)] + \
#                                                     it.loc[:,(path_pr[0],path_pr[1],'Объем кг')] * \
#                                                     it.loc[:,(path_pr[0],path_pr[1],item)] - \
#                                                     it.loc[:,(path_r[0],path_r[1],'Объем кг')] * \
#                                                     it.loc[:,(path_r[0],path_r[1],item)] ) / \
#                                                     (  it.loc[:,(path_ost_nach[0],path_ost_nach[1],'Объем кг')] + it.loc[:,(path_pr[0],path_pr[1],'Объем кг')] - it.loc[:,(path_r[0],path_r[1],'Объем кг')])
        

        # ----------------------------------

        #------------------------------------
        
        
        it.loc[:,(path_ost_kon[0],path_ost_kon[1],'Итого с/c тг/кг')] =  it.loc[:,(path_ost_kon[0],path_ost_kon[1],cols_sum)].sum(axis = 1) 
           
        
        if ind___+1==month:
            template_ = get_current_data(budj_AMD.reset_index().set_index(['Артикул','канал','Наименование','продукт']), global_index, current_month)[current_month].reset_index()[['Артикул','канал','Наименование','продукт']]
            
            return it,template_,mapping

    
#         return it

In [ ]:
ost_AMP_d,template_for_ss_sku,mapping = get_ss_amp(month,
           budj_AMD,
           global_index,
           year_report,
           bolv,
           ost_AMD,
          sebes_amp,
          'Mapping.xlsx',
          'Амортизация.xlsx',
          'Остатки на начало года АМП.xlsx',
          ost_AMP)

In [ ]:
def get_ss_sku(template_for_ss_sku,budj_AMD,ost_AMP_d,ost_AMD,filename,mapping,mpf_file,ukpf_file,month):
    
    cons_table_dict = {} 
#     подготовка затрат АМД
    
    cost_amd =  pd.read_excel(directory+filename,sheet_name='РР АМД')
    cost_amp =  pd.read_excel(directory+filename,sheet_name='РР АМП')
    cost_ukpf =  pd.read_excel(directory+ukpf_file,sheet_name='РР')
    cost_mpf =  pd.read_excel(directory+mpf_file,sheet_name='РР')
    adm_cost_AMP =  pd.read_excel(directory+filename,sheet_name='ОАР АМП')
    adm_cost_AMD =  pd.read_excel(directory+filename,sheet_name='ОАР АМД')
    
    
    def get_mapping_into_cost(aa):
        for ind__ in aa.index:
            if (aa.at[ind__,'Группа'] != aa.at[ind__,'Группа']) == False:
                ind_inp = ind__
            else:
                aa.at[ind__,'map1'] = aa.at[ind_inp,'Группа']
        return aa
    
    
    cost_amd = get_mapping_into_cost(cost_amd)
    
    dict_cost_AMD = rev_to_dict(mapping['Расходы по реализации АМД']['Расходы по реализации АМД'],mapping['Расходы по реализации АМД']['Канал'])
    dict_cost_AMD1 = rev_to_dict(mapping['Статьи затрат']['Статьи затрат'],mapping['Статьи затрат']['Группа затрат, для службы продаж'])
    dict_cost_AMD2 = rev_to_dict(mapping['Статьи затрат']['Статьи затрат'],mapping['Статьи затрат']['Группа затрат, для службы логистики'])
    dict_cost_AMD3 = rev_to_dict(mapping['Статьи затрат']['Статьи затрат'],mapping['Статьи затрат']['Группа затрат, для оставшихся'])
    
    cost_amd['map2'] = cost_amd.apply(lambda x: insert_vpr(x['map1'],dict_cost_AMD,'Канал') ,axis = 1).astype('str')
    
    
    
    def func(x,col,col1,col2): 
        try:
            
            str_  = str(x['map1']).lower()
            if str_.find('продаж')!=-1 or str_.find('проф')!=-1 or str_.find('кейтеринг')!=-1 \
                or str_.find('отп')!=-1 or str_.find('экспорт')!=-1:
                
                return dict_cost_AMD1[x['Служба продаж']][col]
            
            elif str_.find('логистики')!=-1:
                
                return dict_cost_AMD2[x['Служба продаж']][col1]
            
            else:
                
                return dict_cost_AMD3[x['Служба продаж']][col2]
                
        except KeyError:
            pass

    cost_amd['map3'] = cost_amd.apply(lambda x:func(x,
                                                    'Группа затрат, для службы продаж',
                                                    'Группа затрат, для службы логистики',
                                                    'Группа затрат, для оставшихся'
                                                   ),axis = 1)
        
    
    
    
    iter_months = [datetime.datetime(year_report,x,calendar.monthrange(year_report,x)[1],0,0) for x in range(1,13)]
    
    for ind___, current_month in enumerate(iter_months):
        
        print(current_month)
        
        curr_budj = get_current_data(budj_AMD.reset_index().set_index(['Артикул','канал']), global_index, current_month)[current_month].reset_index()
        curr_cost_amp = get_current_data(cost_amp, global_index, current_month)[current_month].sum()
        curr_cost_amd = get_current_data(cost_amd.set_index(['map2','map3']), global_index, current_month)[current_month].reset_index()
        curr_cost_ukpf = get_current_data(cost_ukpf.set_index('Наименование показателя'), global_index, current_month)[current_month].reset_index()
        curr_cost_mpf = get_current_data(cost_mpf.set_index('Наименование показателя'), global_index, current_month)[current_month].reset_index()
        curr_adm_cost_AMP = get_current_data(adm_cost_AMP.set_index('ОАР АМП'), global_index, current_month)[current_month].reset_index()
        curr_adm_cost_AMD = get_current_data(adm_cost_AMD.set_index('ОАР АМД'), global_index, current_month)[current_month].reset_index()
        
        
#         блок продажи
        stock = template_for_ss_sku[['Артикул','канал']].copy()
        
        def func(x,df,col): 
            return df[(df['канал'] == x['канал'])&(df['Артикул'] == x['Артикул'])][col].sum()

        stock['Объем кг'] = stock.apply(lambda x:func(x,curr_budj,'Кол-во') ,axis = 1)
        
        stock['Цена тг/кг'] = stock.apply(lambda x:func(x,curr_budj,'Сумма') ,axis = 1) /stock['Объем кг']
        stock = stock.set_index(['Артикул','канал'])
        
        ml = pd.MultiIndex.from_tuples([ ('Продажи', x) for x in  stock.columns])
        stock.columns = ml
        
#         блок себестоимость

        sebes = template_for_ss_sku[['Артикул','канал']].copy()
        current_rash_AMD = ost_AMD.set_index('Артикул')['Расход'][current_month].copy().reset_index()
        current_rash_AMP = ost_AMP_d.set_index('Артикул')['Расход'][current_month].copy().reset_index()
        
        all_ = current_rash_AMD.append(current_rash_AMP)
        
        cols_sum = []
        for col in all_.set_index('Артикул').columns:
            
            if col!='Объем кг' and col.find('Итого')==-1:
            
                def func(x,df,col__): 
                    return df[df['Артикул'] == x['Артикул']][col__].sum()

                sebes[col] = sebes.apply(lambda x:func(x,all_,col) ,axis = 1)
                cols_sum.append(col)
        
        
        sebes['Итого тг/кг'] = sebes[cols_sum].sum(axis = 1)
        sebes = sebes.set_index(['Артикул','канал'])
        
        
        
        ml = pd.MultiIndex.from_tuples([ ('Себестоимость', x) for x in  sebes.columns])
        sebes.columns = ml
        
        stock_sebes = pd.concat([stock,sebes],axis = 1)
        stock_sebes = stock_sebes.reset_index()
        
        
#         Блок расходы по реализации


# ---------------------------------------------------
        
        def func(x,df): 
            if x['канал'][0]=='ОПТ АМП':
                return curr_cost_amp * ( x['Продажи']['Объем кг'] /  df[df['канал']=='ОПТ АМП']['Продажи']['Объем кг'].sum() )
            else:
                return 0
    
        stock_sebes.loc[:,('Расходы по реализации','РР АМП (ГП)')] = stock_sebes.apply(lambda x:func(x,stock_sebes) ,axis = 1) / \
                                                                    stock_sebes['Продажи']['Объем кг'] * 1000
        
        
# ---------------------------------------------------

        def func(x,df,df1,channel,pr_cost): 
#             if x['канал'][0]=='Кейтеринг':
            prod = x['Продажи']['Объем кг']
            a =  (df1[(df1['map2']==channel) & (df1['map3']==pr_cost)][current_month].sum()/1000) * prod / \
                    df[df['канал']==channel]['Продажи']['Объем кг'].sum()  /  prod * 1000 

            b = prod / ( df['Продажи']['Объем кг'].sum() - df[df['канал']=='АМП пром']['Продажи']['Объем кг'].sum() )  * \
                (df1[(df1['map2']=='на все каналы') & (df1['map3']==pr_cost)][current_month].sum()/1000) / prod * 1000
                
            return a+b
                
        
        stock_sebes.loc[:,('Расходы по реализации','Логистика тг/кг')] = stock_sebes.apply(lambda x:func(x,
                                                                                                   stock_sebes,curr_cost_amd,
                                                                                                   x['канал'][0],
                                                                                                  'Логистика') ,axis = 1)
        
        stock_sebes.loc[:,('Расходы по реализации','Дистрибуция тг/кг')] = stock_sebes.apply(lambda x:func(x,
                                                                                                   stock_sebes,curr_cost_amd,
                                                                                                   x['канал'][0],
                                                                                                  'Дистрибуция') ,axis = 1)
        stock_sebes.loc[:,('Расходы по реализации','Маркетинг тг/кг')] = stock_sebes.apply(lambda x:func(x,
                                                                                                   stock_sebes,curr_cost_amd,
                                                                                                   x['канал'][0],
                                                                                                  'Маркетинг') ,axis = 1)
        
        stock_sebes.loc[:,('Расходы по реализации','Прочее (ЗПП+АМД) тг/кг')] = stock_sebes.apply(lambda x:func(x,
                                                                                                   stock_sebes,curr_cost_amd,
                                                                                                   x['канал'][0],
                                                                                                  'Прочее') ,axis = 1)
        
        def func(x,df,df1,pr_cost,stock_sebes): 
            
            if x['канал'][0]!='АМП пром':
                prod = x['Продажи']['Объем кг']
                a =  prod / (stock_sebes['Продажи']['Объем кг'].sum() - df[df['канал']=='АМП пром']['Продажи']['Объем кг'].sum() ) * \
                    curr_cost_mpf[curr_cost_mpf['Наименование показателя']==pr_cost][current_month].sum() /prod*1000 
                
                b =  prod / (stock_sebes['Продажи']['Объем кг'].sum() - df[df['канал']=='АМП пром']['Продажи']['Объем кг'].sum() ) * \
                    curr_cost_ukpf[curr_cost_ukpf['Наименование показателя']==pr_cost][current_month].sum() /prod*1000 
                
                return a + b
        
        stock_sebes.loc[:,('Расходы по реализации','Прочее (ЗПП+АМД) тг/кг')] =stock_sebes.loc[:,('Расходы по реализации','Прочее (ЗПП+АМД) тг/кг')] + \
                                                                                                    stock_sebes.apply(lambda x:func(x,
                                                                                                   stock_sebes,curr_cost_amd,
                                                                                                  'Прочее',
                                                                                                stock_sebes) ,axis = 1)
        
        stock_sebes.loc[:,('Расходы по реализации','Итого тг/кг')] = stock_sebes.loc[:,('Расходы по реализации','Логистика тг/кг')] + \
                                                                    stock_sebes.loc[:,('Расходы по реализации','Дистрибуция тг/кг')] + \
                                                                    stock_sebes.loc[:,('Расходы по реализации','Маркетинг тг/кг')] + \
                                                                    stock_sebes.loc[:,('Расходы по реализации','Прочее (ЗПП+АМД) тг/кг')]
            
        
        def func(x,df,df2): 
            
            if x['канал'][0]=='АМП пром':
                return 0
            else:
                
                prod = x['Продажи']['Объем кг']
                a =  prod / (stock_sebes['Продажи']['Объем кг'].sum() - df[df['канал']=='АМП пром']['Продажи']['Объем кг'].sum() ) * \
                    df2[current_month].sum() /prod*1000 
                
                return a
            
        
    
        stock_sebes.loc[:,('Адм. расходы','Адм. расходы АМД тг/кг')] = stock_sebes.apply(lambda x:func(x,stock_sebes,curr_adm_cost_AMD) ,axis = 1) 
        
        
        def func(x,df,df2): 
            
            if x['канал'][0]=='ОПТ АМП':
                prod = x['Продажи']['Объем кг']
                a = df2[current_month].sum()   *  (prod/ df[df['канал']=='ОПТ АМП']['Продажи']['Объем кг'].sum() ) 
                return a
                
            else:
                return 0 
                
                
        
        stock_sebes.loc[:,('Адм. расходы','Адм. расходы АМП тг/кг')] = stock_sebes.apply(lambda x:func(x,stock_sebes,curr_adm_cost_AMP) ,axis = 1) /stock_sebes['Продажи']['Объем кг'] * 1000
        
        stock_sebes.loc[:,('Адм. расходы','Итого тг/кг')] = stock_sebes.loc[:,('Адм. расходы','Адм. расходы АМД тг/кг')] + \
                                                            stock_sebes.loc[:,('Адм. расходы','Адм. расходы АМП тг/кг')]
        
        
#         блок сс

        
        stock_sebes.loc[:,('СС','С/с реализованной продукции тг/кг')] = stock_sebes.loc[:,('Себестоимость','Итого тг/кг')]
        
        sum_cols = [x for x in stock_sebes['Себестоимость'].columns if x not in ['Мясосырье, Птицевосдтво, тг/кг','C/c мясосырья, Птицеводство тг/кг','Итого тг/кг']]
        
        stock_sebes.loc[:,('СС','С/с реализованной продукции ДиП тг/кг')] = stock_sebes['Себестоимость'][sum_cols].sum(axis = 1)
        
        stock_sebes.loc[:,('СС','C/c мясосырья, Птицеводство тг/кг')] = stock_sebes.loc[:,('Себестоимость','C/c мясосырья, Птицеводство тг/кг')]
        
        stock_sebes.loc[:,('СС','Полная с/с реализованной продукции тг/кг')] = stock_sebes.loc[:,('Расходы по реализации','Итого тг/кг')] + \
                                                                                stock_sebes.loc[:,('Адм. расходы','Итого тг/кг')] + \
                                                                                stock_sebes.loc[:,('СС','С/с реализованной продукции тг/кг')]
        
        sum_cols_r = [x for x in stock_sebes['Расходы по реализации'] if x not in ['Итого тг/кг']]
        sum_cols_adm = [x for x in stock_sebes['Адм. расходы'] if x not in ['Итого тг/кг']]
        
        stock_sebes.loc[:,('СС','Полная с/с реализованной продукции ДиП тг/кг')] = stock_sebes['Расходы по реализации'][sum_cols_r].sum(axis = 1) + \
                                                                                   stock_sebes['Адм. расходы'][sum_cols_adm].sum(axis = 1) + \
                                                                                    stock_sebes.loc[:,('СС','С/с реализованной продукции ДиП тг/кг')]
        
        stock_sebes.loc[:,('СС','EBITDA по SKU (AKZ) тг/кг')] = stock_sebes.loc[:,('Продажи','Цена тг/кг')] - \
                                                                stock_sebes.loc[:,('СС','Полная с/с реализованной продукции тг/кг')]
        
        stock_sebes.loc[:,('СС','EBITDA margin по SKU (AKZ) тг/кг')] = stock_sebes.loc[:,('СС','EBITDA по SKU (AKZ) тг/кг')] / \
                                                                        stock_sebes.loc[:,('Продажи','Цена тг/кг')]
        
#         Блок пг

        pr_copy = stock_sebes['Продажи'].copy()
        pr_copy['Доходы тыс. тг'] = pr_copy['Объем кг']*pr_copy['Цена тг/кг']
        pr_copy.drop('Цена тг/кг' ,axis = 1, inplace = True)

        a = stock_sebes['Себестоимость'].multiply(stock_sebes['Продажи']['Объем кг'], axis="index")/1000
        c = stock_sebes['Расходы по реализации'].multiply(stock_sebes['Продажи']['Объем кг'], axis="index")/1000
        b = stock_sebes['Адм. расходы'].multiply(stock_sebes['Продажи']['Объем кг'], axis="index")/1000

        cons_pg = pd.concat([pr_copy,a,c,b],axis = 1)

        ml = pd.MultiIndex.from_tuples([ ('Для ПГ', x) for x in  cons_pg.columns])

        cons_pg.columns = ml
        
        stock_sebes = pd.concat([stock_sebes,cons_pg],axis = 1)
        
        cons_table_dict[current_month] = stock_sebes
        
        if ind___+1==month:    
            return cons_table_dict
        
    
#         return template_for_ss_sku.set_index(['Артикул','канал','Наименование','продукт'])

In [ ]:
ss_sku = get_ss_sku(template_for_ss_sku,budj_AMD,ost_AMP_d,ost_AMD,'АМП и АМД.xlsx',mapping,'МПФ.xlsx','УКПФ.xlsx',month)

In [ ]:
ss_sku

In [ ]:
df['Адм. расходы'].multiply(df['Продажи']['Объем кг'], axis="index")/1000

In [ ]:
df['Расходы по реализации'].multiply(df['Продажи']['Объем кг'], axis="index")/1000

In [ ]:
(df['Продажи']['Объем кг'] * df['Адм. расходы']['Адм. расходы АМП тг/кг']).sum()/1000

In [ ]:
aa['Аренда']['Группа затрат, для службы продаж']

In [ ]:
mapping['Расходы по реализации АМД']

In [ ]:
df[(df['map2']=='Кейтеринг') & (df['map3']=='Логистика')]

In [ ]:
df['Расходы по реализации']['Маркетинг'].unique()

In [ ]:
start_date = datetime.datetime(year_report-1, 12, 31, 0, 0)
path_ost_nach = ('Остаток',start_date)

In [ ]:
prod_UKPF[datetime.datetime(2021,1,31)]

In [ ]:
def func(x,df,filter_,current_month): 
    return x[filter_][0]

aa.apply(lambda x:func(x,
                            bb,              
                            'Артикул',
                            datetime.datetime(2021,1,31)
                           ) ,axis = 1) 

In [ ]:
ost_UKPF.set_index('Артикул')['Расход'][datetime.datetime(2021,1,31)]

In [ ]:
ost_AMD[ost_AMD['Артикул']==1080]

In [ ]:
ost_UKPF[ost_UKPF['Артикул']==1080]

In [ ]:
aa = prod_UKPF[datetime.datetime(2021,6,30)]

In [ ]:
a[a['Артикул']==2178]

In [ ]:
from collections import Counter

In [ ]:
c = Counter(a['Артикул'])

In [ ]:
a = a.drop_duplicates(subset=['Артикул'])

In [ ]:
len(  list(a['Артикул']) ) 

In [ ]:
directory_out = 'C:\\Users\\Кедрун Никита\\PycharmProjects\\akz_\\input\\output\\'

In [ ]:
iter_months = [datetime.datetime(year_report,x,calendar.monthrange(year_report,x)[1],0,0) for x in range(1,13)]

In [ ]:
save_iter_month_xlsx(directory_out,prod_UKPF,iter_months,'Себестоимость мяса птицы УКПФ.xlsx')

In [ ]:
save_iter_month_xlsx(directory_out,prod_MPF,iter_months,'Себестоимость мяса птицы МПФ.xlsx')

In [ ]:
save_iter_month_xlsx(directory_out,per_1_ukpf,iter_months,'Первый передел УКПФ.xlsx')

In [ ]:
save_iter_month_xlsx(directory_out,per_1_mpf,iter_months,'Первый передел МПФ.xlsx')

In [ ]:
save_iter_month_xlsx(directory_out,per_2_ukpf,iter_months,'Второй передел УКПФ.xlsx')

In [ ]:
save_iter_month_xlsx(directory_out,per_2_mpf,iter_months,'Второй передел МПФ.xlsx')

In [ ]:
save_iter_month_xlsx(directory_out,per_2_mpf,iter_months,'Второй передел МПФ.xlsx')

In [ ]:
save_iter_month_xlsx(directory_out,ss_sku,iter_months,'Итог сс по SKU.xlsx')

In [ ]:
ost_UKPF.to_excel(directory_out+' Остатки УКПФ.xlsx')
ost_MPF.to_excel(directory_out+' Остатки МПФ.xlsx')

In [ ]:
for date in iter_months:
    prod_UKPF[date].to_excel(directory_out+str(date.month)+'Себестоимость мяса птицы УКПФ.xlsx')

In [ ]:
for date in iter_months:
    prod_MPF[date].to_excel(directory_out+str(date.month)+'Себестоимость мяса птицы МПФ.xlsx')

In [ ]:
for date in iter_months:
    per_1_ukpf[date].to_excel(directory_out+str(date.month)+' Первый передел УКПФ.xlsx')


In [ ]:
for date in iter_months:
    per_1_mpf[date].to_excel(directory_out+str(date.month)+' Первый передел МПФ.xlsx')

In [ ]:
for date in iter_months:
    per_2_ukpf[date].to_excel(directory_out+str(date.month)+' Второй передел УКПФ.xlsx')


In [ ]:
for date in iter_months:
    per_2_mpf[date].to_excel(directory_out+str(date.month)+' Второй передел МПФ.xlsx')

In [ ]:
ost_AMD[ost_AMD['Артикул']==1077]

In [ ]:
import pandas as pd
df = pd.DataFrame({'hhh':[1,2,3]})

In [ ]:
import win32com.client

In [ ]:
win32com.client.Dispatch("V83.COMConnector")

In [ ]:
df = pd.read_excel('C:\\Users\\Кедрун Никита\\test\\АМП и АМД.xlsx',sheet_name='Для СС АМП',header=[0,1])

In [ ]:
df = df.set_index(['№ п/п','Артикул'])

In [ ]:
Production_output =  pd.read_excel(directory_out+'МПФ1.xlsx',sheet_name='Лист1')

In [ ]:
Production_output

In [ ]:
sale_finished_products_UKPF =  pd.read_excel(directory+filename,sheet_name='Продажи ГП')

In [ ]:
[x for x in sale_finished_products_UKPF.columns if sale_finished_products_UKPF[x].dtype== np.float64 ]

In [ ]:
def float_to_decimal(df):
    cols = [x for x in df.columns if df[x].dtype== np.float64 ]
    for col in cols:df[col] = df[col].apply(lambda x:Decimal(x))
    return df

In [ ]:
float_to_decimal(sale_finished_products_UKPF)

In [ ]:
aa = prod_MPF[datetime.datetime(2021,2,28)]

In [ ]:
aa['C/c мясосырья, Птицеводство тг/кг'].sum()

In [ ]:
from decimal import Decimal,InvalidOperation

In [ ]:
Decimal(1000)

In [ ]:
Production_output =  pd.read_excel(directory+filename,sheet_name='Выпуск ГП')

In [ ]:
Production_output[Production_output['Артикул']==1066][[datetime.datetime(2021,2,28)]].values

In [ ]:
Decimal(1002.05393768258)/Decimal(2404.45000000) *1000

In [ ]:
from decimal import Decimal, ROUND_HALF_EVEN

In [ ]:
ost_MPF.to_excel(directory_out+' ddd.xlsx')

In [ ]:
'ddd'.encode().decode('utf-8', 'ignore')

In [ ]:
a = "\n\"\"\"\nTraceback (most recent call last):\n  File \"C:\\Users\\\u041a\u0435\u0434\u0440\u0443\u043d \u041d\u0438\u043a\u0438\u0442\u0430\\AppData\\Local\\Programs\\Python\\Python39\\lib\\multiprocessing\\pool.py\", line 125, in worker\n    result = (True, func(*args, **kwds))\n  File \"C:\\Users\\\u041a\u0435\u0434\u0440\u0443\u043d \u041d\u0438\u043a\u0438\u0442\u0430\\AppData\\Local\\Programs\\Python\\Python39\\lib\\multiprocessing\\pool.py\", line 48, in mapstar\n    return list(map(*args))\n  File \"C:\\Users\\\u041a\u0435\u0434\u0440\u0443\u043d \u041d\u0438\u043a\u0438\u0442\u0430\\PycharmProjects\\project_reports_akz\\modules\\margin_report\\module_ssmp.py\", line 68, in get_ssmp_ukpf\n    per_1_UKPF = retype_index(per_1_UKPF, 'int', '\u0410\u0440\u0442\u0438\u043a\u0443\u043b')\n  File \"C:\\Users\\\u041a\u0435\u0434\u0440\u0443\u043d \u041d\u0438\u043a\u0438\u0442\u0430\\PycharmProjects\\project_reports_akz\\modules\\margin_report\\builtin_functions.py\", line 29, in retype_index\n    for col in df.columns:\nAttributeError: 'int' object has no attribute 'columns'\n\"\"\""

In [ ]:
a.replace('\n','')

In [ ]:
import re

In [ ]:
result = re.findall(r'.py', a)


In [ ]:
assert 0 != 0,'ОТсутствует месяц'

In [ ]:
for month in iter_months:
    

In [ ]:
iter_months = [datetime.datetime(year_report,x,calendar.monthrange(year_report,x)[1],0,0) for x in range(1,6+1)]

In [ ]:
for month in iter_months:
    assert iter_months.count(month)==1,'Отсутствует столбец' 

In [ ]:
a = 'првв'

In [ ]:
isinstance(iter_months[0],datetime.datetime)